In [5]:
import numpy as np
import pandas as pd

import json

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from geopy.geocoders import Nominatim

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import plotly.io as pio
pio.renderers.default = 'browser'

import warnings
warnings.filterwarnings('ignore')

# style.use('fivethirtyeight')

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

pd.set_option('display.max_colwidth', -1)

rand_state=1000

# Energy Related Data Sets

## 1. Energy Efficiency Projects

The City’s Long-Term Sustainability Plan, PlaNYC, calls for City government to reduce greenhouse gas emissions produced by municipal operations 30% by 2017. A large part of this effort is retrofitting municipal buildings to become more energy efficient. This dataset lists energy efficiency projects at City buildings that are complete or in progress. Projects are listed by address, building name, and agency, and the dataset includes the estimated reduction in tons of GHG emissions associated with the project.

https://data.cityofnewyork.us/Environment/Energy-Efficiency-Projects/h3qk-ybvt


In [21]:
df_Energy_Efficiency_Projects=pd.read_csv('data/all/Energy_Efficiency_Projects.csv')
print(df_Energy_Efficiency_Projects.shape)
df_Energy_Efficiency_Projects.head()

(584, 19)


,PrimaryProjectType,SubType,ProjectStatus,StartDateActual,FinishDateActual,ProjectSiteName,ProjectSiteAddress,Acronym,CO2e (MT) Calculated,Postcode,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,HVAC System Upgrade,NaN,Completed,NaN,NaN,NaN,NaN,BPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lighting System Upgrade,NaN,Completed,NaN,NaN,NaN,NaN,NaN,76.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Lighting System Upgrade,NaN,Completed,NaN,NaN,NaN,NaN,NaN,37.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Comprehensive,Retrofit,Design,NaN,NaN,NaN,NaN,DCAS,2060.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Other,Retro-commissioning,Proposed,NaN,NaN,Brooklyn Supreme Court,292-360 Adams Strrt,DCAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_Energy_Efficiency_Projects['PrimaryProjectType'].unique().tolist()

['HVAC System Upgrade',
 'Lighting System Upgrade',
 'Comprehensive',
 'Other',
 'Energy Audit',
 nan]

## 2. NYC Municipal Building Energy Benchmarking ResultsCity Government


This is a list of New York City municipal buildings over 10,000 square feet by borough, block, lot, and agency, identifying each building’s energy intensity (kBtu/sq. ft.), Portfolio Manager benchmarking rating, where available, and the total GHG emissions for the calendar years 2010 - 2013.

https://data.cityofnewyork.us/City-Government/NYC-Municipal-Building-Energy-Benchmarking-Results/vvj6-d5qx

In [22]:
df_Municipal_Building_Energy_Benchmarking=pd.read_csv('data/all/NYC_Municipal_Building_Energy_Benchmarking_Results.csv')
print(df_Municipal_Building_Energy_Benchmarking.shape)
df_Municipal_Building_Energy_Benchmarking.head()

(3388, 20)


,Borough,Block,Lot,BIN,Building,Agency,2010 Rating *,2010 Source EUI (kBtu/sq.ft.),2010 GHG Emissions Intensity (kgCO2e/ft²) †,2011 Rating *,2011 Source EUI (kBtu/sq.ft.),2011 GHG Emissions Intensity (kgCO2e/ft²) †,2012 Rating *,2012 Source EUI (kBtu/sq.ft.),2012 GHG Emissions Intensity (kgCO2e/ft²) †,2013 Rating *,2013 Source EUI (kBtu/sq.ft.),2013 GHG Emissions Intensity (kgCO2e/ft²) †,EPAPMBenchmarkedAs,Campus Name
0,2,2383,12,2001505,1332 Fulton Ave D C C Inc,ACS,N/A,180.6,4.9,NaN,155.9,4.3,NaN,157.7,4.3,NaN,164.8,4.5,Individual building,NaN
1,1,93,1,1062652,ACS Administrative Site,ACS,57,239.0,6.2,55,242.1,6.3,59,228.3,5.9,64.0,217.9,5.6,Individual building,NaN
2,3,4484,13,3098733,ACS Administrative Site,ACS,16,286.6,8.6,19,268.3,7.9,20,258.6,7.6,24.0,252.1,7.5,Space within building,NaN
3,2,4101,1,2044743,ACS Bronx Field Office,ACS,7,316.2,8.9,14,269.6,7.2,12,273.0,7.4,12.0,282.8,7.9,Individual building,NaN
4,3,2022,18,3057913,Administration Site,ACS,21,313.6,9.8,31,272.4,8.4,35,257.9,7.5,37.0,256.1,7.6,Individual building,NaN


## 3. DCAS Managed Building Energy Usage

City Building Energy Usage Data.

https://data.cityofnewyork.us/City-Government/DCAS-Managed-Building-Energy-Usage/ubdi-jgw2

In [23]:
df_City_Building_Energy_Usage=pd.read_csv('data/all/DCAS_Managed_Building_Energy_Usage.csv')
print(df_City_Building_Energy_Usage.shape)
df_City_Building_Energy_Usage.head()

(55, 17)


,Borough Number,Building Name,Building Address,Borough,State,Postcode,Block,Lot,"FY15 Energy Usage (MMBTU) [Utility energy, excluding Fuel Oil",Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1,Manhattan Municipal Building,1 Centre St,Manhattan,New York,10007,121,1,92117,40.713001,-74.004181,1.0,1.0,29.0,1001394.0,1.001210e+09,Chinatown
1,5,Staten Island Borough Hall,10 Richmond Ter,Staten Island,New York,10301,7,1,11385,40.642197,-74.075201,1.0,49.0,3.0,5000063.0,5.000070e+09,West New Brighton-New Brighton-St. George
2,1,Manhattan Criminal Court,100 Centre Street,Manhattan,New York,10013,167,1,126511,40.716084,-74.001425,1.0,1.0,29.0,1079000.0,1.001670e+09,Chinatown
3,1,100 Gold St,100 Gold St,Manhattan,New York,10038,94,25,64582,40.710353,-74.004007,1.0,1.0,1501.0,1001289.0,1.000940e+09,Battery Park City-Lower Manhattan
4,5,Staten Island Family Court,100 Richmond Ter,Staten Island,New York,10301,9,22,1429,40.644935,-74.077230,1.0,49.0,7.0,5000090.0,5.000090e+09,West New Brighton-New Brighton-St. George


## 4. LL87 Energy Audit Data

Energy audit data for privately owned buildings larger than 50,000 ft2, submitted by property owners in compliance with Local Law 87 of 2009. This data includes equipment inventory and recommended Energy Conservation Measures (ECMs) for base building systems: the building envelope, HVAC systems, elevators and escalators, domestic hot water supply, and electrical and lighting systems. This data does not include equipment owned by tenants or used for industrial processes within the building. This data represents energy audits collected between 2012 and 2018 using the City’s original, MS Excel-based, Energy Efficiency Report tool.
    
https://data.cityofnewyork.us/Environment/LL87-Energy-Audit-Data/au6c-jqvf

In [26]:
df_Private_Energy_Audit= pd.read_csv('data/all/LL87_data_through_2018.csv',
                                    encoding = 'unicode_escape')
print(df_Private_Energy_Audit.shape)
df_Private_Energy_Audit.head()

(6175, 1017)


,BBL,Submittal Information_Submission Information_date,Submittal Information_Submission Information_early compliance,Submittal Information_Submission Information_filing status,Submittal Information_Lot Information_borough,Submittal Information_Lot Information_block,Submittal Information_Lot Information_lot,Submittal Information_Lot Information_# of buildings on lot,Submittal Information_Building Information_BIN,Submittal Information_Building Information_Address,Submittal Information_Building Information_Zip code,Submittal Information_Comment,Team Information_Audit Team Information_Audit Report Date,Team Information_Comment,Team Information_BINs,Building Characteristics_Gross Floor Area,Building Characteristics_Conditioned Area Heated only,Building Characteristics_Conditioned Area cooled only,"Building Characteristics_Conditioned Area, heated & cooled",Building Characteristics_Total conditioned Area,Building Characteristics_# of above grade floors,Building Characteristics_# of below grade floors,Building Characteristics_Year of construction/substantial Rehabilitation,Simple Building_1st Check Box,Simple Building_2nd Check Box,Simple Building_3rd Check Box,Simple Building_4th Check Box,Simple Building_5th Check Box,Simple Building_6th Check Box,Simple Building_7th Check Box,Space Types_Bank/Financial Institution,Space Types_Courthouse,Space Types_Data Center,Space Types_Hospital,Space Types_Hotel,Space Types_House of Worship,Space Types_K-12 School,Space Types_Medical Office,Space Types_Office,Space Types_Residence Hall/Dorm,Space Types_Retail,Space Types_Senior Care Facility,Space Types_Supermarket/Grocery,Space Types_Warehouse,Space Types_Multifamily,Space Types_Other,Metering Configuration_Multi Tenant?,Metering Configuration_Tenants Directly metered_Electric,Metering Configuration_Tenants Directly metered_Gas,Metering Configuration_Tenants sub-metered by building owner_Electric,Metering Configuration_Tenants sub-metered by building owner_Gas,Metering Configuration_Tenants not directly metered or sub-metered_Electric,Metering Configuration_Tenants not directly metered or sub-metered_Gas,Metering Configuration_Tenants Directly metered_Electric.1,Metering Configuration_Tenants Directly metered_Gas.1,Metering Configuration_Tenants sub-metered by building owner_Electric.1,Metering Configuration_Tenants sub-metered by building owner_Gas.1,Metering Configuration_Tenants not directly metered or sub-metered_Electric.1,Metering Configuration_Tenants not directly metered or sub-metered_Gas.1,Energy system Configuration_Shared Energy system or meters for Multi buildings on a single lot,Energy system Configuration_Shared Energy system or meters for Multi buildings on multiple lots,Energy system Configuration_Shared Electric meter,Energy system Configuration_Shared chilled water,Energy system Configuration_Shared gas meter,Energy system Configuration_Shared oil,Shared utility steam met,Benchmarking_Portfolio Manager Building ID,Benchmarking_Year benchmarked,Benchmarking_facility type,Benchmarking_if other specify,Benchmarking_rating,Benchmarking_weather normalized source EUI,Benchmarking_site energy intensity,Benchmarking_Total GHG Emissions,Benchmarking_Energy Cost Index,Comment,BINs,Heating Component_Heating System 1_Heating System Type,"Heating Component_Heating System 1_If other, specify",Heating Component_Heating System 1_Quantity,Heating Component_Heating System 1_Equipment Tag#,Heating Component_Heating System 1_Spaces served,Heating Component_Heating System 1_Year Installed,Heating Component_Heating System 1_Fuel sources,"Heating Component_Heating System 1_If other, specify.1",Heating Component_Heating System 1_Controls,"Heating Component_Heating System 1_If other, specify.2",Heating Component_Heating System 2_Heating System Type,"Heating Component_Heating System 2_If other, specify",Heating Component_Heating System 2_Quantity,Heating Component_Heating System 2_Equipment Tag#,Heating Component_Heating System 2_Spaces served,Heatin

## 5. Energy Usage From DOE Buildings

Energy data from a select portfolio of City-owned buildings (DOE)

https://data.cityofnewyork.us/Environment/Energy-Usage-From-DOE-Buildings/mq6n-s45c

In [29]:
df_City_owned_buildings_Energy_Usage= \
    pd.read_csv('data/all/Energy_Usage_From_DOE_Buildings.csv')
print(df_City_owned_buildings_Energy_Usage.shape)
df_City_owned_buildings_Energy_Usage.head()

(153, 62)


,Building Address,Borough,Measurement,-,_,(FY 2009) 7/1/2008,Aug-08,Sep-08,Oct-08,Nov-08,Dec-08,Jan-09,Feb-09,Mar-09,Apr-09,May-09,Jun-09,(FY 2010)7/1/2009,Aug-09,Sep-09,Oct-09,Nov-09,Dec-09,Jan-10,Feb-10,Mar-10,Apr-10,May-10,Jun-10,(FY 2011)7/1/2010,Aug-10,Sep-10,Oct-10,Nov-10,Dec-10,Jan-11,Feb-11,Mar-11,Apr-11,May-11,Jun-11,(FY 2012)7/1/2011,Aug-11,Sep-11,Oct-11,Nov-11,Dec-11,Jan-12,Feb-12,Mar-12,Apr-12,May-12,Jun-12,Postcode,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,512 West 212th St [Shorac Kappock School]\n,1,Electricity Demand (KW),144.00,68.0,148.00,158.00,166.00,180.00,168.00,166.00,166.0,162.00,142.00,146.00,146.00,90.00,148.0,152.00,170.00,178.0,184.00,190.0,188.00,172.00,158.00,154.00,136.00,64.00,140.00,156.0,166.00,178.00,178.00,178.00,172.00,164.00,154.00,170.00,156.00,82.00,58.00,168.00,164.00,162.00,166.00,160.0,156.00,146.00,146.00,156.00,NaN,NaN,10034,40.867826,-73.917059,12,10,303,1064871.0,1.022290e+09,Marble Hill-Inwood
1,512 West 212th St [Shorac Kappock School]\n,1,Electricity Usage (KWH),30400.00,31000.0,46200.00,45200.00,51800.00,58400.00,56600.00,52200.00,55200.0,44200.00,48600.00,47400.00,34200.00,30600.00,44800.0,52400.00,54200.00,63200.0,60800.00,63200.0,58000.00,48200.00,50800.00,52200.00,33000.00,28600.00,42600.00,52800.0,53800.00,65200.00,55600.00,59600.00,58000.00,48200.00,49600.00,52000.00,31400.00,20600.00,9800.00,91600.00,58800.00,53400.00,57600.00,58600.0,50400.00,43800.00,47400.00,53600.00,NaN,NaN,10034,40.867826,-73.917059,12,10,303,1064871.0,1.022290e+09,Marble Hill-Inwood
2,512 West 212th St [Shorac Kappock School]\n,1,Gas (Therms),109.00,112.0,305.00,2298.00,7666.00,13710.00,16197.00,12846.00,10322.0,4660.00,1138.00,348.00,150.00,112.00,243.0,2835.00,5873.00,14903.0,15662.00,16322.0,8197.00,2966.00,1298.00,275.00,106.00,93.00,193.00,1645.0,6448.00,16825.00,16787.00,14045.00,10629.00,5373.00,831.00,339.00,124.00,115.00,273.00,778.00,6037.00,8103.00,13972.00,12298.0,6866.00,3781.00,1410.00,302.00,NaN,NaN,10034,40.867826,-73.917059,12,10,303,1064871.0,1.022290e+09,Marble Hill-Inwood
3,512 West 212th St [Shorac Kappock School]\n,1,Total Usage (mmBTUs),114.65,117.0,188.18,384.07,943.39,1570.32,1812.87,1462.76,1220.6,616.85,279.67,196.57,131.72,115.64,177.2,462.34,772.28,1706.0,1773.71,1847.9,1017.65,461.11,303.18,205.66,123.23,106.91,164.69,344.7,828.42,1905.03,1868.46,1607.91,1260.85,701.81,252.38,211.37,119.57,81.81,60.75,390.43,804.38,992.55,1593.79,1429.8,858.61,527.59,302.77,213.14,NaN,NaN,10034,40.867826,-73.917059,12,10,303,1064871.0,1.022290e+09,Marble Hill-Inwood
4,232 East 103rd St [Roberto Clemente School]\n,1,Electricity Demand (KW),123.20,150.4,150.40,124.80,136.00,144.00,144.00,144.00,142.4,142.40,145.60,140.80,128.00,134.40,148.8,128.00,128.00,128.0,126.40,126.4,124.80,121.60,140.80,160.00,153.60,139.20,156.80,148.8,134.40,134.40,134.40,134.40,129.60,131.20,124.80,182.40,171.20,145.60,174.40,169.60,139.20,132.80,132.80,132.8,128.00,158.40,134.40,187.20,NaN,NaN,10029,40.788952,-73.944538,11,8,164,1052384.0,1.016520e+09,East Harlem South


## 6. Energy Demand Response Program

The Demand Response Program provides for the reduction of electric usage by participating energy customers during heat waves, emergencies, and other periods of peak demand in order to reduce the strain on New York City's electric grid, avoid service disruptions, and reduce greenhouse gas emissions. In return, enrolled customers receive credits based on the amount of energy reduced, measured in kilowatts (kW).

https://data.cityofnewyork.us/Environment/Energy-Demand-Response-Program/vavg-cg3g

In [31]:
df_Energy_Demand_Response_Program= \
    pd.read_csv('data/all/Energy_Demand_Response_Program.csv')
print(df_Energy_Demand_Response_Program.shape)
df_Energy_Demand_Response_Program.head()

(26, 7)


,Time Period,Agency/Organization,Agency/Organization (Acronym),Total Commitment (kW),NYISO SCR Commitment (kW),Con Ed DRLP Commitment (kW),Con Ed CSRP Commitment (kW)
0,Winter 2017/2018 + Summer 2018,Brooklyn Public Library,BPL,100,100,50.0,100.0
1,Winter 2017/2018 + Summer 2018,City University of New York,CUNY,8309,8099,5425.0,4825.0
2,Winter 2017/2018 + Summer 2018,Department of Cultural Affairs - American Museum of Natural History,DCLA - American Museum of Natural History,400,400,200.0,200.0
3,Winter 2017/2018 + Summer 2018,Department of Cultural Affairs - Bronx Zoo,DCLA - Bronx Zoo,200,200,100.0,NaN
4,Winter 2017/2018 + Summer 2018,Department of Cultural Affairs - Brooklyn Museum,DCLA - Brooklyn Museum,200,200,50.0,50.0


## 7. NYC Municipal Building Energy Benchmarking Results (2014)City Government

This is a list of New York City municipal buildings over 10,000 square feet by borough, block, lot, and agency, identifying each building’s energy intensity (kBtu/sq. ft.), Portfolio Manager benchmarking rating, where available, and the total GHG emissions for the calendar years 2010 - 2014.

https://data.cityofnewyork.us/City-Government/NYC-Municipal-Building-Energy-Benchmarking-Results/hpid-63r5


In [33]:
df_NYC_Municipal_Building_Energy_Benchmarking_Results__2014_= \
    pd.read_csv('data/all/NYC_Municipal_Building_Energy_Benchmarking_Results__2014_.csv')
print(df_NYC_Municipal_Building_Energy_Benchmarking_Results__2014_.shape)
df_NYC_Municipal_Building_Energy_Benchmarking_Results__2014_.head()

(2477, 12)


,Borough,Block,Lot,BIN,Building,Agency,2010 Score *,2010Source EUI (kBtu/ft²)*,2010 GHG Emissions Intensity (kgCO2e/ft²)*,2014 Score *,2014 Source EUI (kBtu/ft²)*,2014 GHG Emissions Intensity (kgCO2e/ft²)*
0,1.0,93.0,1.0,1001280.0,ACS Administrative Site (Leased),ACS,57,239,6.2,76,219.5,5.8
1,1.0,350.0,8.0,1004240.0,Escuela Hispana Montessori #2 (Leased),ACS,Not Available,162.5,5.1,Not Available,151,5.3
2,1.0,479.0,1.0,1007156.0,Chinese Community Concern Parents,ACS,Not Available,289.4,8.6,Not Available,294.1,9.6
3,1.0,2014.0,36.0,1060189.0,Lutheran Social Service of NY #11 (Leased),ACS,Not Available,237.1,7.5,Not Available,220,7.1
4,1.0,2032.0,17.0,1060503.0,Lutheran Social Service of NY #13 (Leased),ACS,Not Available,193.6,5.1,Not Available,208.5,5.5


## 8. Energy Efficiency Portfolio Standard (EEPS) Program Estimated Energy Savings Data

The Energy Efficiency Portfolio Standard (EEPS) Program encourages cost-effective electric and natural gas energy efficiency across New York State. New York State’s utilities and the New York State Energy Research and Development Authority (NYSERDA) administer energy efficiency programs to achieve energy efficiency savings. EEPS energy efficiency programs provide technical services, information and customer incentives to encourage customers in implementing energy efficiency measures. This data includes the list of energy efficiency programs and the estimated energy savings reported for each program.

https://data.ny.gov/Energy-Environment/Energy-Efficiency-Portfolio-Standard-EEPS-Program-/vqmi-95p7

In [36]:
df_Energy_Efficiency_Portfolio_Standard__EEPS__Program_Estimated_Energy_Savings_Data= \
    pd.read_csv('data/all/Energy_Efficiency_Portfolio_Standard__EEPS__Program_Estimated_Energy_Savings_Data.csv')
print(df_Energy_Efficiency_Portfolio_Standard__EEPS__Program_Estimated_Energy_Savings_Data.shape)
df_Energy_Efficiency_Portfolio_Standard__EEPS__Program_Estimated_Energy_Savings_Data.head()

(2712, 7)


,Program Administrator,Program Name,Fuel Type,Sector,Net Fuel Savings Acquired,Fuel Savings Unit,Cumulative Net Energy Savings Acquired as of Date
0,Central Hudson Gas & Electric Corporation,Expanded Residential HVAC Program,Electric,Residential,0.00,MWh,09/30/2019
1,Central Hudson Gas & Electric Corporation,Home Energy Reporting,Electric,Residential,74836.28,MWh,09/30/2019
2,Central Hudson Gas & Electric Corporation,Home Energy Reporting,Gas,Residential,97050.76,Dth,09/30/2019
3,Central Hudson Gas & Electric Corporation,Mid-Size Commercial Business Program,Electric,Commercial,35711.02,MWh,09/30/2019
4,Central Hudson Gas & Electric Corporation,Residential Appliance Recycling Program,Electric,Residential,19562.37,MWh,09/30/2019


## 9. Office Energy Intensity
This is an Excel file with a list of New York City municipal buildings over 10,000 square feet by borough, block, lot, and agency, identifying each building’s energy intensity (kBtu/sq. ft.), Portfolio Manager benchmarking rating, where available, and the total GHG emissions for the calendar year 2010.

https://nycopendata.socrata.com/City-Government/Office-Energy-Intensity/bmet-43b8

In [38]:
df_Office_Energy_Intensity= \
    pd.read_csv('data/all/Office_Energy_Intensity.csv')
print(df_Office_Energy_Intensity.shape)
df_Office_Energy_Intensity.head()

(3388, 9)


,Borough,Block,Lot,BIN,Building,Agency,2010 Rating *,2010 Source EUI (kBtu/sq.ft.),2010 GHG Emissions Intensity (kgCO2e/ft²) †
0,2,2383,12,2001505,1332 Fulton Ave D C C Inc,ACS,N/A,180.6,4.9
1,1,93,1,1062652,ACS Administrative Site,ACS,57,239.0,6.2
2,3,4484,13,3098733,ACS Administrative Site,ACS,16,286.6,8.6
3,2,4101,1,2044743,ACS Bronx Field Office,ACS,7,316.2,8.9
4,3,2022,18,3057913,Administration Site,ACS,21,313.6,9.8


## 10. Projected Citywide Energy Cost

This dataset summarizes citywide energy cost projections. Dollar amounts are rounded to millions. This dataset is updated bi-annually,

https://data.cityofnewyork.us/City-Government/Projected-Citywide-Energy-Cost/tyv9-j3ti

In [40]:
df_Projected_Citywide_Energy_Cost= \
    pd.read_csv('data/all/Projected_Citywide_Energy_Cost.csv')
print(df_Projected_Citywide_Energy_Cost.shape)
df_Projected_Citywide_Energy_Cost.head()

(125, 4)


,PUBLICATION DATE,TYPE,FISCAL YEAR,AMOUNT
0,20200416,Gasoline,2020,71
1,20200416,Fuel Oil,2020,53
2,20200416,HPD-In Rem / DAMP,2020,10
3,20200416,HPD-Emergency Repairs,2020,4
4,20200416,"Heat, Light & Power",2020,709


## 11. Energy and Water Data Disclosure for Local Law 84 2021 (Data for Calendar Year 2020)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/usc3-8zwd

In [42]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2021__Data_for_Calendar_Year_2020_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2021__Data_for_Calendar_Year_2020_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2021__Data_for_Calendar_Year_2020_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2021__Data_for_Calendar_Year_2020_.head()

(28067, 250)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Year Ending,"NYC Borough, Block and Lot (BBL)",NYC Building Identification Number (BIN),Address 1,City,Postcode,Primary Property Type - Self Selected,Primary Property Type - Portfolio Manager-Calculated,National Median Reference Property Type,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Construction Status,Number of Buildings,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,National Median ENERGY STAR Score,Target ENERGY STAR Score,Reason(s) for No Score,ENERGY STAR Certification - Year(s) Certified (Score),ENERGY STAR Certification - Eligibility,ENERGY STAR Certification - Last Approval Date,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),National Median Site EUI (kBtu/ft²),Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),Source Energy Use (kBtu),Weather Normalized Source Energy Use (kBtu),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Propane Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Natural Gas Use (therms),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kWh),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems (kWh),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Annual Maximum Demand (Meter Name (Meter ID)),Green Power - Onsite (kWh),Green Power - Offsite (kWh),Green Power - Onsite and Offsite (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Total GHG Emissions Intensity (kgCO2e/ft²),Direct GHG Emissions Intensity (kgCO2e/ft²),Indirect GHG Emissions (Metric Tons CO2e),Net Emissions (Metric Tons CO2e),Indirect GHG Emissions Intensity (kgCO2e/ft²),National Median Total GHG Emissions (Metric Tons CO2e),eGRID Output Emissions Rate (kgCO2e/MBtu),Avoided Emissions - Onsite Green Power (Metric Tons CO2e),Avoided Emissions - Offsite Green Power (Metric Tons CO2e),Avoided Emissions - Onsite and Offsite Green Power (Metric Tons CO2e),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,LEED US Project ID,Adult Education - Gross Floor Area (ft²),Ambulatory Surgical Center - Gross Floor Area (ft²),Automobile Dealership - Gross Floor Area (ft²),Bank Branch - Gross Floor Area (ft²),Barracks- Gross Floor Area (ft²),College/University - Gross Floor Area (ft²),Convention Center - Gross Floor Area (ft²),Courthouse - Gross Floor Area (ft²),Data Center - Gross Floor Area (ft²),Data Center - UPS System Redundancy,Data Center - Cooling Equipment Redundancy,Data Center - IT Energy Configuration,...,Multifamily Housing - Number of Bedrooms,"Multifamily Housing - Number of Bedrooms Density (Number per 1,000 sq ft)",Multifamily Housing - Number of Laundry Hookups in All Units,Multifamily Housing - Number of Laundry Hookups in Common Area(s),Multifamily Housing - Number of Residential

## 12. Wind Energy Projects: 2004-2015

Table lists existing and proposed major wind projects in NYS, their locations, relative electric generating potential (project nameplate size), and project status (proposed, in-construction, or operational).

https://data.ny.gov/Energy-Environment/Wind-Energy-Projects-2004-2015/jmxa-iz8m

In [44]:
df_Wind_Energy_Projects__2004_2015= \
    pd.read_csv('data/all/Wind_Energy_Projects__2004-2015.csv')
print(df_Wind_Energy_Projects__2004_2015.shape)
df_Wind_Energy_Projects__2004_2015.head()

(38, 8)


,PSC Case Number,Project Name,Project Developer,Location Town,Location County,No. of Turbines,Project MW,Status
0,14-F-0490,Cassadaga Wind,Everpower,"Arkwright, Charlotte, Cherry Creek, Stockton",Chatauqua,NaN,126,Article 10 PSS filing
1,NaN,Arkwright Summit,EDPR,Arkwright,Chautauqua,47.0,79.9,SEQRA pending
2,NaN,Ripley- Westfield Wind Farm,Babcock & Brown Renewables,Ripley & Westfield,Chautauqua,83.0,125,Withdrawn
3,06-E-0216,Noble Altona Windpark,Noble Environmental Power,Altona,Clinton,68.0,102,Operational
4,15-F-0377,Bull Run Wind Farm,"Bull Run Energy, LLC - Invenergy","Altona, Clinton, Ellenburg & Mooers",Clinton,NaN,300,Article 10 PIP filing


## 13. Natural Gas Consumption by ZIP Code - 2010

2010 Natural Gas consumption in therms and GJ, by ZIP code, building type, and utility company.

https://data.cityofnewyork.us/Environment/Natural-Gas-Consumption-by-ZIP-Code-2010/uedp-fegm

In [48]:
df_Natural_Gas_Consumption_by_ZIP_Code_2010= \
    pd.read_csv('data/all/Natural_Gas_Consumption_by_ZIP_Code_-_2010.csv')
print(df_Natural_Gas_Consumption_by_ZIP_Code_2010.shape)
df_Natural_Gas_Consumption_by_ZIP_Code_2010.head()

(1015, 5)


,Zip Code,Building type (service class,Consumption (therms),Consumption (GJ),Utility/Data Source
0,10300,Commercial,470.0,50.0,National Grid
1,10335,Commercial,647.0,68.0,National Grid
2,10360,Large residential,33762.0,3562.0,National Grid
3,11200,Commercial,32125.0,3389.0,National Grid
4,11200,Institutional,3605.0,380.0,National Grid


## 14. Energy Efficiency Completed Projects: Beginning 1987

The Energy Efficiency programs of the New York Power Authority provide energy-efficiency improvements, with no up-front costs, to public schools and other government facilities. From start to finish, the Power Authority works with facility managers to identify, design and install new lighting and motors, as well as upgrades to heating, ventilation and air-conditioning systems. We try to address all energy efficiency improvements in a single, comprehensive effort. This data set contains information on energy efficiency projects completed since 1987. The data set is updated in a quarterly basis to reflect new data as projects are implemented. The information includes project location, customer name, project name, total cost, and energy efficiency benefits, including energy reduction (electric, natural gas, oil) and greenhouse gas emissions reductions.




In [49]:
df_Energy_Efficiency_Completed_Projects__Beginning_1987= \
    pd.read_csv('data/all/Energy_Efficiency_Completed_Projects__Beginning_1987.csv')
print(df_Energy_Efficiency_Completed_Projects__Beginning_1987.shape)
df_Energy_Efficiency_Completed_Projects__Beginning_1987.head()

(2606, 31)


,Year completed,Date Completed,Project Code,Project Name,Regional Economic Development Council,NYISO Zone,NY State County,NY Region,Customer,Agency,Project category,Facility Description,Number of Facilities,Total Project Cost,First Year Bill Savings,Demand Reduction (KW),Electric Energy Reduction (kWh),Gas Savings (Therms),Oil Savings (Gallons),Steam Savings (M Lbs),Digester Gas Savings (CCF),Coal Savings (Tons),Equivalent Barrels of Oil Saved,Reduction in Greenhouse Gas Emissions (Tons),Renewable Energy (KW),Address,Address Ext,City,State,Zip,Project or Program description
0,1987,12/31/1987,CH79001,Watt Busters - SKANEATELES,Central New York,C - Central,ONONDAGA,Central,"SKANEATELES, TOWN OF",NaN,Energy Efficiency,Municipal Utility,1.0,17396,633.0,7.0,16000.0,0.0,0.0,0.0,NaN,0.0,37.06897,1.842592,0.0,NaN,NaN,NaN,NY,NaN,Home energy audits and weatherization services were provided to residential customers served by NYPA’s municipal and cooperative system customers. This program was funded by NYPA financing.
1,1987,12/31/1987,CH79002,Watt Busters - SHERRILL,Mohawk Valley,E - Mohawk Valley,ONEIDA,Central,"SHERRILL, CITY OF",NaN,Energy Efficiency,Municipal Utility,1.0,41098,1729.0,30.0,77000.0,0.0,0.0,0.0,NaN,0.0,178.10350,8.867474,0.0,NaN,NaN,NaN,NY,NaN,Home energy audits and weatherization services were provided to residential customers served by NYPA’s municipal and cooperative system customers. This program was funded by NYPA financing.
2,1987,12/31/1987,CH10725,Watt Busters - GROTON,Southern Tier,C - Central,TOMPKINS,Central,"GROTON, VILLAGE OF",NaN,Energy Efficiency,Municipal Utility,1.0,65284,9175.0,101.0,265000.0,0.0,0.0,0.0,NaN,0.0,612.75870,30.517930,0.0,NaN,NaN,NaN,NY,NaN,Home energy audits and weatherization services were provided to residential customers served by NYPA’s municipal and cooperative system customers. This program was funded by NYPA financing.
3,1988,12/31/1988,CH79003,Watt Busters - BERGEN,Finger Lakes,A - West,GENESEE,Western,"BERGEN, VILLAGE OF",NaN,Energy Efficiency,Municipal Utility,1.0,20049,1182.0,9.0,21000.0,0.0,0.0,0.0,NaN,0.0,48.62069,2.418402,0.0,NaN,NaN,NaN,NY,NaN,Home energy audits and weatherization services were provided to residential customers served by NYPA’s municipal and cooperative system customers. This program was funded by NYPA financing.
4,1988,12/31/1988,CH10723,Watt Busters - SPENCERPORT,Finger Lakes,B - Genesee,MONROE,Central,"SPENCERPORT, VILLAGE OF",NaN,Energy Efficiency,Municipal Utility,1.0,174264,26128.0,346.0,880000.0,0.0,0.0,0.0,NaN,0.0,2035.00000,101.342600,0.0,NaN,NaN,NaN,NY,NaN,Home energy audits and weatherization services were provided to residential customers served by NYPA’s municipal and cooperative system customers. This program was funded by NYPA financing.


## 15. Energy and Water Data Disclosure for Local Law 84 2020 (Data for Calendar Year 2019)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/wcm8-aq5w

In [51]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2020__Data_for_Calendar_Year_2019= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2020__Data_for_Calendar_Year_2019.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2020__Data_for_Calendar_Year_2019.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2020__Data_for_Calendar_Year_2019.head()

(29084, 254)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Year Ending,"NYC Borough, Block and Lot (BBL)",NYC Building Identification Number (BIN),Address 1,Address 2,City,Postcode,Primary Property Type - Self Selected,Primary Property Type - Portfolio Manager-Calculated,National Median Reference Property Type,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Construction Status,Number of Buildings,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,National Median ENERGY STAR Score,ENERGY STAR Certification - Year(s) Certified,ENERGY STAR Certification - Last Approval Date,ENERGY STAR Certification - Eligibility,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),National Median Site EUI (kBtu/ft²),% Difference from National Median Site EUI,Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),% Difference from National Median Source EUI,Source Energy Use (kBtu),Weather Normalized Source Energy Use (kBtu),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Kerosene Use (kBtu),Propane Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Natural Gas Use (therms),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kWh),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kWh),Electricity Use – Generated from Onsite Renewable Systems (kWh),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Annual Maximum Demand (Meter Name (Meter ID)),Green Power - Onsite (kWh),Green Power - Offsite (kWh),Green Power - Onsite and Offsite (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Total GHG Emissions Intensity (kgCO2e/ft²),Direct GHG Emissions Intensity (kgCO2e/ft²),Indirect GHG Emissions (Metric Tons CO2e),Net Emissions (Metric Tons CO2e),Indirect GHG Emissions Intensity (kgCO2e/ft²),National Median Total GHG Emissions (Metric Tons CO2e),eGRID Output Emissions Rate (kgCO2e/MBtu),Avoided Emissions - Onsite Green Power (Metric Tons CO2e),Avoided Emissions - Offsite Green Power (Metric Tons CO2e),Avoided Emissions - Onsite and Offsite Green Power (Metric Tons CO2e),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,CoStar Property ID,LEED US Project ID,Adult Education - Gross Floor Area (ft²),Ambulatory Surgical Center - Gross Floor Area (ft²),Automobile Dealership - Gross Floor Area (ft²),Bank Branch - Gross Floor Area (ft²),Barracks- Gross Floor Area (ft²),College/University - Gross Floor Area (ft²),Convention Center - Gross Floor Area (ft²),Courthouse - Gross Floor Area (ft²),Data Center - Gross Floor Area (ft²),...,Multifamily Housing - Number of Bedrooms,"Multifamily Housing - Number of Bedrooms Density (Number per 1,000 sq ft)",Multifamily Housing - Number of Laundry Hookups in All Units,Multifamily Housing - Number of Laundry Hookups in Common Area(s),Multifamily Housing - Number of Residential Living Units in a High-Rise Buildin

## 16. Value of Energy Cost Savings Program Savings for Businesses - FY2018

Total estimated annual dollar value of utility discounts for businesses approved for ECSP benefits during the fiscal year

https://data.cityofnewyork.us/City-Government/Value-of-Energy-Cost-Savings-Program-Savings-for-B/ukdt-xm28

In [53]:
df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2018= \
    pd.read_csv('data/all/Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2018.csv')
print(df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2018.shape)
df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2018.head()

(568, 27)


,Company ID,Company Name,company contact,company email,company phone,Address,City,State,Postcode,Industry,Industry Description,Company Type,Current fulltime,Job create,Job retain,Effective Date,Total sav,Savings from begining receiving benefits,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,3410,"139 ACA Realty, Inc.",Eitan Chandally,barbara@dial7.com,7.187076e+09,43-23 35th Street,Long Island City,NY,11101,Commercial,Limousine Service,ICIP,125.0,NaN,NaN,04/07/2008,8314.92,123975.24,QUEENS,40.745706,-73.929565,2.0,26.0,179.0,4003160.0,4.002220e+09,Hunters Point-Sunnyside-West Maspeth
1,3531,"141 Lake Avenue Realty c/o JR Produce, Inc.",Josef Raz,jrproduce@gmail.com,7.183709e+09,141 Lake Avenue,Staten Island,NY,10303,Wholesale/Warehouse/Distribution,Dist. of prepacked salads,ICIP,6.0,NaN,NaN,12/08/2009,6010.71,47512.89,STATEN IS,40.633153,-74.150999,1.0,49.0,239.0,5146740.0,5.011610e+09,Mariner's Harbor-Arlington-Port Ivory-Graniteville
2,3465,14-10 123rd Street LLC,Danica/Ivan Drazic,ddrazic@atjelectrical.com,7.183210e+09,14-10 123rd Street,College Point,NY,11356,Commercial,Electrical Parts Mfg.,ICIP,35.0,NaN,NaN,03/04/2011,2946.05,21322.89,QUEENS,40.785144,-73.844833,7.0,19.0,929.0,4098344.0,4.040850e+09,College Point
3,3800,183 Lorriane Street LLC,Tom Sapienza,tsapienza@KLCNY.com,2.128406e+09,183 Lorraine Street,Brooklyn,NY,11231,Wholesale/Warehouse/Distribution,Commercial Storage facility,ICIP,9.0,NaN,NaN,11/06/2015,48981.51,105016.49,BROOKLYN,40.673106,-74.002300,6.0,38.0,53.0,3336622.0,3.005720e+09,Carroll Gardens-Columbia Street-Red Hook
4,3482,"21st Century Optics, Inc.",Ralph Woythaler,rwoythaler@21st centuryoptics.com,7.183922e+09,47-00 33rd Street,Lond Island City,NY,11101,Manufacturing,Eye glasses,Tenant,130.0,NaN,NaN,01/07/2009,36400.37,215757.20,QUEENS,40.742386,-73.932148,2.0,26.0,199.0,4003447.0,4.002520e+09,Hunters Point-Sunnyside-West Maspeth


## 17. Value of Energy Cost Savings Program Savings for Businesses - FY2020

Total estimated annual dollar value of utility discounts for businesses approved for ECSP benefits during the fiscal year.

https://data.cityofnewyork.us/City-Government/Value-of-Energy-Cost-Savings-Program-Savings-for-B/bug8-9f3g

In [55]:
df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2020= \
    pd.read_csv('data/all/Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2020.csv')
print(df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2020.shape)
df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2020.head()

(450, 26)


,Company Name,company contact,company email,company phone,Address,City,State,Postcode,Industry,Industry descr,Company Type,Current fulltime,Job created,Job retain,Effective Date,Total Savings,Savings from beginning receiving benefits,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1 State Street,David Neiburg,eskoblo@energyspec.com,7186779077,1114 Avenue J,Brooklyn,NY,11230,Commercial,35 story building,LMEP,NaN,NaN,NaN,03/04/2005,0.00,349.78,Brooklyn,40.624784,-73.964920,314.0,44.0,534.0,3179818.0,3.067130e+09,Midwood
1,100 Church LLC,Edward A. Carroll,NaN,2125663000,100 Church Street,NY,NY,10007,Commercial,21 Floors,LMEP,NaN,NaN,NaN,04/02/2008,136578.20,4423698.05,New York,40.712968,-74.009490,101.0,1.0,21.0,1001409.0,1.001250e+09,SoHo-TriBeCa-Civic Center-Little Italy
2,100 Wall Investments,Jerry Branam,NaN,2122487160,96 Wall Street,New York,NY,10005,Landlord,NaN,LMEP,1238.0,NaN,NaN,01/05/2015,302959.40,1814542.62,New York,40.705052,-74.007199,101.0,1.0,7.0,1000875.0,1.000380e+09,Battery Park City-Lower Manhattan
3,1-10 Bush Terminal Owner LP,Andrew Kimball,akimball@industrycity.com,3474178429,"220 36th St, Suite 2A",Brooklyn,NY,11232,Landlord,Special Eligible Premises for eligible businesses,ECSP,25.0,NaN,NaN,02/03/2020,2159.67,6755.31,Brooklyn,40.656061,-74.007334,307.0,38.0,2.0,3398687.0,3.006950e+09,Sunset Park West
4,"120 Broadway Acquisitions JV, LLC (Ldmk)",Carl Letich,clettich@silvprop.com,2124064020,120 Broadway,New York,NY,10271,Commercial,NaN,LMEP,2699.0,NaN,NaN,06/30/2017,922407.32,3293055.08,New York,40.708546,-74.011041,101.0,1.0,7.0,1001026.0,1.000478e+09,Battery Park City-Lower Manhattan


## 18. Jobs Created or Retained by Energy Cost Savings Program - FY2018

Total number of full-time employees at the time company was approved for ECSP benefits

https://data.cityofnewyork.us/City-Government/Jobs-Created-or-Retained-by-Energy-Cost-Savings-Pr/j46e-fnm6

In [57]:
df_Jobs_Created_or_Retained_by_Energy_Cost_Savings_Program_FY2018= \
    pd.read_csv('data/all/Jobs_Created_or_Retained_by_Energy_Cost_Savings_Program_-_FY2018.csv')
print(df_Jobs_Created_or_Retained_by_Energy_Cost_Savings_Program_FY2018.shape)
df_Jobs_Created_or_Retained_by_Energy_Cost_Savings_Program_FY2018.head()

(59, 15)


,Applicant Name,Address,Postcode,EMP,Invenstment,Savings (Annual),Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,"Computer Services Group, Inc.","37-24 24th Street, Suite 340, Queens, NY",10018,21,NaN,"$2,994.22",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Gotham Greens Gowanus, LLC","184-60 Jamaica Avenue, Queens, NY",11423,33,"$3,357,824.00","$54,545.00",QUEENS,40.709522,-73.774514,12.0,27.0,444.0,4220567.0,4.103520e+09,Jamaica
2,New York Fragrance,"600 Gulf Avenue, Staten Island",10314,7,"$2,215,900.00",TBD,STATEN IS,40.614257,-74.179839,2.0,50.0,29102.0,5000000.0,5.017600e+09,New Springville-Bloomfield-Travis
3,"Chelsea Craft Brewing Co., Inc.","463 E. 173rd Street, Bronx, NY",10457,10,"$1,620,000.00","$4,848.00",BRONX,40.841778,-73.901638,3.0,15.0,165.0,2124550.0,2.029060e+09,Claremont-Bathgate
4,"S & L Aerospace Metals, LLC","120-12 28th Avenue, Queens, NY",11354,87,"$827,511.00","$27,471.00",QUEENS,40.773145,-73.847648,7.0,19.0,907.0,4100711.0,4.043170e+09,College Point


## 19. Steam Consumption by ZIP Code - 2010

2010 steam consumption in mlbs and Mg, by ZIP code and building type.

https://data.cityofnewyork.us/Environment/Steam-Consumption-by-ZIP-Code-2010/2pmt-skyq

In [59]:
df_Steam_Consumption_by_ZIP_Code_2010= \
    pd.read_csv('data/all/Steam_Consumption_by_ZIP_Code_-_2010.csv')
print(df_Steam_Consumption_by_ZIP_Code_2010.shape)
df_Steam_Consumption_by_ZIP_Code_2010.head()

(102, 5)


,ZIP,Building type (service class),Consumption (mlbs),Consumption (Mg),Utility/Data Source
0,10048,Commercial,72117,32712,ConEd
1,"10007(40.713630519433, -74.009131383706)",Commercial,9050,4105,ConEd
2,"10038(40.709359291898, -74.003071862376)",Commercial,12944,5871,ConEd
3,"10021(40.769205327664, -73.959183941086)",Commercial,253883,115159,ConEd
4,"10038(40.709359291898, -74.003071862376)",Commercial,238716,108280,ConEd


## 20. Oil Usage ( Select City Owned Buildings)

Oil Usage (select City Owned Buildings), oil, city, data, statistics,heating, energy,heat

https://data.cityofnewyork.us/City-Government/Oil-Usage-Select-City-Owned-Buildings-/whux-iuiu

In [61]:
df_Oil_Usage___Select_City_Owned_Buildings_= \
    pd.read_csv('data/all/Oil_Usage___Select_City_Owned_Buildings_.csv')
print(df_Oil_Usage___Select_City_Owned_Buildings_.shape)
df_Oil_Usage___Select_City_Owned_Buildings_.head()

(65, 17)


,BIN Number,Building,SqFt,Electric,2006 Natural Gas Usage,2006 Fuel Oil #2- Delivery,2006 Fuel Oil #4 - Delivery,2006 Fuel Oil #6 - Delivery,2006 Steam usage,2011 Electric usage,2011 Natural Gas usage,2011 Fuel Oil #2 B5 - Delivery,2011 Fuel Oil #2- Delivery,2011 Fuel Oil #4 - Delivery,2011 Fuel Oil #6 - Delivery,2011 Steam usage,Types
0,2009911,Bronx Bergen Building,"* 90,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1740.0,NaN,NaN,Gas Primary/FO Secondary
1,1001289,Office Building 100 Gold St,"* 594,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4660.0,1497.0,NaN,NaN,NaN,Gas Primary/FO Secondary
2,1001835,Clocktower Building,"* 369,376",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ConEd Steam
3,2099027,Bronx Concourse Plaza,"* 231,190",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gas Only
4,1078613,City Planning Building,"* 101,150",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ConEd Steam


## 21. Energy Prices Per Physical Unit By Sector: Beginning 1970

New York Energy Prices presents retail energy price data. Energy prices are provided by fuel type in nominal dollars per specified physical unit for the residential, commercial, industrial, and transportation sectors. This section includes a column in the price table displaying gross domestic product (GDP) price deflators for converting nominal (current year) dollars to constant (real) dollars. To convert nominal to constant dollars, divide the nominal energy price by the GDP price deflator for that particular year. Historical petroleum, electricity, coal, and natural gas prices were compiled primarily from the Energy Information Administration.

How does your organization use this dataset? What other NYSERDA or energy-related datasets would you like to see on Open NY? Let us know by emailing OpenNY@nyserda.ny.gov.

https://data.ny.gov/Energy-Environment/Energy-Prices-Per-Physical-Unit-By-Sector-Beginnin/cxu4-yemb
    
    

In [63]:
df_Energy_Prices_Per_Physical_Unit_By_Sector__Beginning_1970= \
    pd.read_csv('data/all/Energy_Prices_Per_Physical_Unit_By_Sector__Beginning_1970.csv')
print(df_Energy_Prices_Per_Physical_Unit_By_Sector__Beginning_1970.shape)
df_Energy_Prices_Per_Physical_Unit_By_Sector__Beginning_1970.head()

(188, 13)


,Year,GDP Deflator,Sector,Coal ($/Ton),Distillate (cents/gallon),Residual ($/barrel),Kerosene (cents/gallon),Propane (cents/gallon),Natural Gas ($/Mcf),Electricity (cents/kWh),Wood ($/Cord),Motor Gasoline (cents/gallon),Jet Fuel (cents/gallon)
0,2016,1.0000,Commercial,NaN,157.30,38.35,179.15,115.17,6.18,14.45,NaN,NaN,NaN
1,2016,1.0000,Industrial,100.05,154.69,38.35,152.28,107.13,5.92,6.03,NaN,NaN,NaN
2,2016,1.0000,Residential,NaN,227.23,NaN,179.15,247.33,10.85,17.58,114.4,NaN,NaN
3,2016,1.0000,Transportation,NaN,240.55,35.21,NaN,NaN,NaN,12.05,NaN,218.14,128.12
4,2015,0.9875,Commercial,NaN,199.07,49.23,224.78,122.48,6.86,15.31,NaN,NaN,NaN


## 22. Value of Energy Cost Savings Program Savings for Businesses - FY2019

Total estimated annual dollar value of utility discounts for businesses approved for ECSP benefits during the fiscal year.

https://data.cityofnewyork.us/City-Government/Value-of-Energy-Cost-Savings-Program-Savings-for-B/va74-3m6c

In [65]:
df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2019= \
    pd.read_csv('data/all/Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2019.csv')
print(df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2019.shape)
df_Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2019.head()

(544, 26)


,Company Name,company contact,company email,company phone,Address,City,State,Zip,Industry,Industry descr,Company Type,Current fulltime,Job create,Job retain,Effective Date,Total Savings,Savings from beginning receiving benefits,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,100 Church LLC,Edward A. Carroll,NaN,2125663000,100 Church Street,NY,NY,10007,Commercial,21 Floors,LMEP,NaN,NaN,NaN,04/02/2008,415074.57,4287119.85,MANHATTAN,40.712968,-74.009490,101.0,1.0,21.0,1001409.0,1.001250e+09,SoHo-TriBeCa-Civic Center-Little Italy
1,100 Wall Investments,Jerry Branam,NaN,2122487160,96 Wall Street,New York,NY,10005,Landlord,NaN,LMEP,1238.0,NaN,NaN,01/05/2015,285278.88,1290839.09,MANHATTAN,40.705052,-74.007199,101.0,1.0,7.0,1000875.0,1.000380e+09,Battery Park City-Lower Manhattan
2,"120 Broadway Acquisitions JV, LLC (Ldmk)",Carl Letich,clettich@silvprop.com,2124064020,120 Broadway,New York,NY,10271,Commercial,NaN,LMEP,2699.0,NaN,NaN,06/30/2017,918063.14,1723157.93,MANHATTAN,40.708546,-74.011041,101.0,1.0,7.0,1001026.0,1.000478e+09,Battery Park City-Lower Manhattan
3,"120 Wall Street, LLC",Jason McCalla,jmccalla@silvprop.com,2125517391,120 Wall Street,New York,NY,10005,Commercial,Back Offices,LMEP,958.0,NaN,NaN,07/02/2015,374799.79,1474907.67,MANHATTAN,40.704380,-74.006427,101.0,1.0,7.0,1000871.0,1.000370e+09,Battery Park City-Lower Manhattan
4,125 Broad Unit A LLC,Sarah A. Parnes,NaN,2126724759,125 Broad Street,NY,NY,10004,Commercial,NaN,LMEP,NaN,NaN,NaN,02/02/2006,27777.80,780401.30,MANHATTAN,40.702112,-74.011249,101.0,1.0,9.0,1000006.0,1.000058e+09,Battery Park City-Lower Manhattan


## 23. Energy and Water Data Disclosure for Local Law 84 2018 (Data for Calendar Year 2017)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4t62-jm4m

In [68]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2018__Data_for_Calendar_Year_2017_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2018__Data_for_Calendar_Year_2017_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2018__Data_for_Calendar_Year_2017_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2018__Data_for_Calendar_Year_2017_.head()

(34686, 256)


,Property Id,Year Ending,Generation Date,ACCOUNT_ID,Property Id 1,Property Name,Parent Property Id,Parent Property Name,"NYC Borough, Block and Lot (BBL)",NYC Building Identification Number (BIN),Address 1,Address 2,City,County,Postcode,Primary Property Type - Self Selected,Primary Property Type - Portfolio Manager-Calculated,National Median Reference Property Type,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,ENERGY STAR Certification - Year(s) Certified,ENERGY STAR Certification - Last Approval Date,ENERGY STAR Certification - Eligibility,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),National Median Site EUI (kBtu/ft²),% Difference from National Median Site EUI,Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),National Median Site Energy Use (kBtu),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),% Difference from National Median Source EUI,Source Energy Use - Adjusted to Current Year (kBtu),Source Energy Use (kBtu),Weather Normalized Source Energy Use (kBtu),National Median Source Energy Use (kBtu),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Kerosene Use (kBtu),Propane Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems (kWh),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Annual Maximum Demand (Meter Name (Meter ID)),Green Power - Onsite (kWh),Green Power - Offsite (kWh),Avoided Emissions - Onsite Green Power (Metric Tons CO2e),Avoided Emissions - Offsite Green Power (Metric Tons CO2e),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),National Median Total GHG Emissions (Metric Tons CO2e),Estimated Data Flag - Electricity (Grid Purchase),Estimated Data Flag - Natural Gas,Estimated Data Flag - Fuel Oil (No. 2),Estimated Data Flag - Fuel Oil (No. 4),Estimated Data Flag - Fuel Oil (No. 5 and No. 6),Estimated Data Flag - District Steam,eGRID Output Emissions Rate (kgCO2e/MBtu),Net Emissions (Metric Tons CO2e),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,CoStar Property ID,LEED US Project ID,Ambulatory Surgical Center - Gross Floor Area (ft²),Automobile Dealership - Gross Floor Area (ft²),Bank Branch - Gross Floor Area (ft²),Bank Branch - Number of Computers,College/University - Gross Floor Area (ft²),College/University - Number of Computers,...,Multifamily Housing - Percent That Can Be Cooled,Multifamily Housing - Percent That Can Be Heated,Multifamily Housing - Resident Population Type,"Multifamily Housing - Number of Bedrooms Density (Number per 1,000 ft²)","Multifamily Housing - Total Number of Residential Living Units Density (Number per 1,000 ft²)",Multifamily Housing - Number of Residential Living Units in a High-Rise Building (10 or more Stories),"Multifamily Housing - Number of Residential Living Units in a High-Rise Bui

## 24. Jobs Created by Energy Cost Savings Program Savings for Businesses - FY2019

Jobs Created by Energy Cost Savings Program Savings for Businesses

https://data.cityofnewyork.us/City-Government/Jobs-Created-by-Energy-Cost-Savings-Program-Saving/xsd9-52hv

In [72]:
df_Jobs_Created_by_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2019= \
    pd.read_csv('data/all/Jobs_Created_by_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2019.csv')
print(df_Jobs_Created_by_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2019.shape)
df_Jobs_Created_by_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2019.head()

(48, 17)


,No.,APPLICANT NAME,Address,Zip Code,EMP,TOTAL,Savings (Annual),INVESTMENT,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1,The Dime Savings Bank of Williamsburg,"300 Cadman Plaza West, Brooklyn, NY",11201,151,NaN,7354.0,NaN,BROOKLYN,40.695198,-73.990869,302.0,33.0,502.0,3001937.0,3.002390e+09,Brooklyn Heights-Cobble Hill
1,2,VaynerMedia LLC,"21-09 Borden Avenue, Queens, NY",11101,100,NaN,11462.0,NaN,QUEENS,40.740383,-73.949562,402.0,26.0,1.0,4000531.0,4.000690e+09,Hunters Point-Sunnyside-West Maspeth
2,3,"Wild Edibles, Inc.","740 Barry Street, Bronx, NY",10474,28,279.0,16293.0,"$1,343,000.00",BRONX,40.812618,-73.894712,202.0,17.0,93.0,2005902.0,2.027360e+09,Hunts Point
3,4,"North Brooklyn Industrial Center aka Evergreen, Inc. @ 164 Cook Street","164 Cook Street, Brooklyn, NY",11206,0,NaN,NaN,"$230,237.00",BROOKLYN,40.702880,-73.936405,301.0,34.0,485.0,3071626.0,3.031230e+09,Bushwick South
4,5,Allison Eden Studios (164) Tenant in SEP,"164 Cook Street, Brooklyn, NY",11206,6,NaN,1221.0,NaN,BROOKLYN,40.702880,-73.936405,301.0,34.0,485.0,3071626.0,3.031230e+09,Bushwick South


## 25. DCAS Managed Building Fuel Usage

City Building Fuel Usage Data.

https://data.cityofnewyork.us/City-Government/DCAS-Managed-Building-Fuel-Usage/56u5-n9sa
    

In [74]:
df_DCAS_Managed_Building_Fuel_Usage= \
    pd.read_csv('data/all/DCAS_Managed_Building_Fuel_Usage.csv')
print(df_DCAS_Managed_Building_Fuel_Usage.shape)
df_DCAS_Managed_Building_Fuel_Usage.head()

(55, 17)


,Borough Number,Building Name,Building Address,Borough,State,Postcode,Block,Lot,Fuel Oil Usage (MMBTU),Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1,Manhattan Municipal Building,1 Centre Street,Manhattan,New York,10007,121,1,0.000,40.713001,-74.004181,1.0,1.0,29.0,1001394.0,1.001210e+09,Chinatown
1,5,Staten Island Borough Hall,10 Richmond Terrace,Staten Island,New York,10301,7,1,1900.398,40.642197,-74.075201,1.0,49.0,3.0,5000063.0,5.000070e+09,West New Brighton-New Brighton-St. George
2,1,Manhattan Criminal Court,100 Centre Street,Manhattan,New York,10013,167,1,17678.628,40.716084,-74.001425,1.0,1.0,29.0,1079000.0,1.001670e+09,Chinatown
3,1,100 Gold,100 Gold Street,Manhattan,New York,10038,94,25,3557.226,40.710353,-74.004007,1.0,1.0,1501.0,1001289.0,1.000940e+09,Battery Park City-Lower Manhattan
4,5,Staten Island Family Court,100 Richmond Terrace,Staten Island,New York,10301,9,22,0.000,40.644935,-74.077230,1.0,49.0,7.0,5000090.0,5.000090e+09,West New Brighton-New Brighton-St. George


## 26. Energy and Water Data Disclosure for Local Law 84 2019 (Data for Calendar Year 2018)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4tys-3tzj

In [76]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2019__Data_for_Calendar_Year_2018= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2019__Data_for_Calendar_Year_2018.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2019__Data_for_Calendar_Year_2018.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2019__Data_for_Calendar_Year_2018.head()

(25245, 253)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Year Ending,"NYC Borough, Block and Lot (BBL)",NYC Building Identification Number (BIN),Address 1,Address 2,City,County,Postcode,Primary Property Type - Self Selected,Primary Property Type - Portfolio Manager-Calculated,National Median Reference Property Type,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,ENERGY STAR Certification - Year(s) Certified,ENERGY STAR Certification - Last Approval Date,ENERGY STAR Certification - Eligibility,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),National Median Site EUI (kBtu/ft²),% Difference from National Median Site EUI,Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),% Difference from National Median Source EUI,Source Energy Use - Adjusted to Current Year (kBtu),Source Energy Use (kBtu),Weather Normalized Source Energy Use (kBtu),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Kerosene Use (kBtu),Propane Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems (kWh),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Annual Maximum Demand (Meter Name (Meter ID)),Green Power - Onsite (kWh),Green Power - Offsite (kWh),Avoided Emissions - Onsite Green Power (Metric Tons CO2e),Avoided Emissions - Offsite Green Power (Metric Tons CO2e),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),National Median Total GHG Emissions (Metric Tons CO2e),Estimated Data Flag - Electricity (Grid Purchase),Estimated Data Flag - Natural Gas,Estimated Data Flag - Fuel Oil (No. 2),Estimated Data Flag - Fuel Oil (No. 4),Estimated Data Flag - Fuel Oil (No. 5 and No. 6),Estimated Data Flag - District Steam,eGRID Output Emissions Rate (kgCO2e/MBtu),Net Emissions (Metric Tons CO2e),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,CoStar Property ID,LEED US Project ID,Ambulatory Surgical Center - Gross Floor Area (ft²),Automobile Dealership - Gross Floor Area (ft²),Bank Branch - Gross Floor Area (ft²),Bank Branch - Number of Computers,College/University - Gross Floor Area (ft²),College/University - Number of Computers,Convention Center - Gross Floor Area (ft²),Courthouse - Gross Floor Area (ft²),Data Center - Gross Floor Area (ft²),Data Center - PDU Input Meter (kWh),Data Center - PDU Output Meter (kWh),...,Multifamily Housing - Percent That Can Be Heated,Multifamily Housing - Resident Population Type,"Multifamily Housing - Number of Bedrooms Density (Number per 1,000 ft²)","Multifamily Housing - Total Number of Residential Living Units Density (Number per 1,000 ft²)",Multifamily Housing - Number of Residential Living Units in a High-Rise Building (10 or more Stories),"Multifamily Housing - Number of Residential Living Units

## 27. NYC Covered Building's List (2019)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.
    
https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2019-/tphr-wtjd

In [78]:
df_NYC_Covered_Building_s_List__2019_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2019_.csv')
print(df_NYC_Covered_Building_s_List__2019_.shape)
df_NYC_Covered_Building_s_List__2019_.head()

(20556, 20)


,10 Digit BBL,Required to report automated water data from DEP?,Borocode,Block,Lot,Number of Buildings,Tax Class,Building Class,Street Number,Street Name,Postcode,Borough,DOF Square Footage,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1000047501,No,1,4,7501,1,2,R0,1,WATER STREET,10038.0,Manhattan,2542563,40.702444,-74.012742,101.0,1.0,9.0,1000005.0,Battery Park City-Lower Manhattan
1,1000050010,No,1,5,10,1,4,O4,115,BROAD STREET,10004.0,Manhattan,1016406,40.702486,-74.011357,101.0,1.0,9.0,1000007.0,Battery Park City-Lower Manhattan
2,1000057501,No,1,5,7501,1,2,R0,125,BROAD STREET,10004.0,Manhattan,1354691,40.702112,-74.011249,101.0,1.0,9.0,1000006.0,Battery Park City-Lower Manhattan
3,1000080051,Yes,1,8,51,1,4,H2,6,WATER STREET,10004.0,Manhattan,102407,40.702744,-74.012201,101.0,1.0,9.0,1090472.0,Battery Park City-Lower Manhattan
4,1000087501,Yes,1,8,7501,1,2,R0,39,WHITEHALL STREET,0.0,Manhattan,169061,40.703037,-74.012969,101.0,1.0,9.0,1087700.0,Battery Park City-Lower Manhattan


## 28. Energy and Water Data Disclosure for Local Law 84 2012 (Data for Calendar Year 2011)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/k7nh-aufb

In [80]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2012__Data_for_Calendar_Year_2011_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2012__Data_for_Calendar_Year_2011_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2012__Data_for_Calendar_Year_2011_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2012__Data_for_Calendar_Year_2011_.head()

(12400, 210)


,Building ID (ESPM),BBL10,Facility Name,Rating,BIN (Unique Building Identifier),Address 1,Address 2,City,Postcode,County,Total Floor Space (Sq Ft),Year Built,Facility Type,Last Modified Date,Service and Product Provider,Metering Configuration,Eligibility for ENERGY STAR,ENERGY STAR Application Status,Year(s) Labeled,Approval Date - Last ENERGY STAR,Current Rating (1-100),Target Rating (1-100),Energy Use Alerts,Space Use Alerts,Current Direct GHG Emissions (MtCO2e),Current Indirect GHG Emissions (MtCO2e),Current Total GHG Emissions (MtCO2e),Emissions Factor (kg CO2e/MBtu),Current Energy Period Ending Date,Current Water Period Ending Date,Current Total On-Site Renewable Electric Use (kWh),Percent of Electricity from On-Site Renewable (%),Total Avoided GHG Emissions from Green Power (MtCO2e),Total On-Site Electric Generation (kWh),Total Green Power Purchased (MWh),Total Renewable Energy Sold to Grid (kWh),Current Site Energy Intensity (kBtu/Sq Ft),Current Total Site Energy Use (kBtu),National Median Site EUI (kBtu/Sq Ft),Target Site Energy Intensity (kBtu/Sq Ft),Weather Normalized Site EUI (kBtu/Sq Ft),Electricity Use (kBtu),Electricity Use (kWh),Natural Gas Use (kBtu),Natural Gas Use (therms),District Steam Use (kBtu),Fuel Oil 2 Use (kBtu),Fuel Oil 4 Use (kBtu),Fuel Oil 56 Use (kBtu),Diesel 2 Use (kBtu),Chilled Water - Absorption Use (kBtu),Chilled Water - Electric Driven Use (kBtu),Chilled Water - Engine Driven Use (kBtu),Chilled Water - Other (kBtu),District Hot Water (kBtu),Temporary Data Flag - Electricity (Grid Purchase),Temporary Data Flag - Electricity (On-Site Solar),Temporary Data Flag - Electricity (On-Site Wind),Temporary Data Flag - Natural Gas,Temporary Data Flag - District Steam,Temporary Data Flag - Fuel Oil (No 2),Temporary Data Flag - Fuel Oil (No 4),Temporary Data Flag - Fuel Oil (No 5 and No 6),Temporary Data Flag - Diesel,Temporary Data Flag - District Hot Water,Temporary Data Flag - District Chilled Water,% Difference from National Median Site Energy per Square Foot (%,Current Source Energy Intensity (kBtu/Sq Ft),Current Total Source Energy Use (kBtu),Current Weather Normalized Source Energy Intensity (kBtu/Sq Ft,National Median Source EUI (kBtu/Sq Ft),% Difference from National Median Source Energy per Square Foot,Bank/Financial Institution- Gross Floor Area (Sq Ft),Bank/Financial Institution- Office Air-Conditioned,Bank/Financial Institution- PC Density,Bank/Financial Institution- Weekly operating hours,Bank/Financial Institution- Workers Density,Bank/Financial Institution- Workers on Main Shift,Hospital- Gross Floor Area (Sq Ft),"Hospital- Laboratory? (Y=1, N=0)","Hospital- Laundry Facility? (Y=1, N=0)",Hospital- Maximum Number of Floors,Hospital- Number of Buildings,Full-time Equivalent (FTE) Workers,Number of Staffed Beds,Number of MRI Machines,Hotel- Gross Floor Area (Sq Ft),Hotel- Floor area of full-service spas,Hotel- Floor area of gym/fitness center,Hotel- Average Occupancy (%),Hotel- Number of Rooms,Hotel- Room Density,Hotel- Onsite Laundry (short tons/year),Hotel- Quantity of laundry processed on-site annually,Hotel- Percent Cooled,"Hotel- Presence of cooking facilities? (Y=1, N=0)",Hotel- Workers Density,House of Worship- Gross Floor Area (Sq Ft),House of Worship- PC Density,House of Worship- Weekly operating hours,...,Medical Office- Percent Cooled,Medical Office- Weekly operating hours,Multifamily Home- Gross Floor Area (Sq Ft),Multifamily Home- Dishwasher hookups in all buildings,Multifamily Home- Laundry in common area,Multifamily Home- Laundry in each unit,"Multifamily Home- Government Subsidized Housing? (Y=1, N=0)",Multifamily Home- Number of Floors,Multifamily Home- Number of units,Multifamily Home- Percent Cooled,Multifamily Home- Percent of gross floor area that is common spa,Multifamily Home- Total Number of Bedrooms,Multifamily Home- Primary Hot Water Fuel Type (for units),Multifamily Home- Resident Population Type,Office- Gross Floor Area (Sq Ft),Office- Office- Air-Conditioned,Offic

## 29. NYC Covered Building's List (2020)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.

https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2020-/eukp-v77b
    

In [82]:
df_NYC_Covered_Building_s_List__2020_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2020_.csv')
print(df_NYC_Covered_Building_s_List__2020_.shape)
df_NYC_Covered_Building_s_List__2020_.head()

(24071, 20)


,BBL,Required to report automated water data from DEP?,Borocode,Block,Lot,Number of Buildings,Tax Class,Building Class,Street Number2,Street Name,Postcode,Borough,DOF Square Footage,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1000020002,No,1,2,2,1,4,Y7,10,SOUTH STREET,10004,Manhattan,"158,197",40.701695,-74.011631,101.0,1.0,9.0,1000003.0,Battery Park City-Lower Manhattan
1,1000047501,No,1,4,7501,1,2,R0,1,WATER STREET,10038,Manhattan,"2,542,563",40.702444,-74.012742,101.0,1.0,9.0,1000005.0,Battery Park City-Lower Manhattan
2,1000050010,No,1,5,10,1,4,O4,115,BROAD STREET,10004,Manhattan,"1,016,406",40.702486,-74.011357,101.0,1.0,9.0,1000007.0,Battery Park City-Lower Manhattan
3,1000057501,No,1,5,7501,1,2,R0,125,BROAD STREET,10004,Manhattan,"1,354,691",40.702112,-74.011249,101.0,1.0,9.0,1000006.0,Battery Park City-Lower Manhattan
4,1000080039,Yes,1,8,39,1,4,H3,32,PEARL STREET,10004,Manhattan,"46,724",40.703235,-74.012421,101.0,1.0,9.0,1078968.0,Battery Park City-Lower Manhattan


## 30. Energy and Water Data Disclosure for Local Law 84 2011 (Data for Calendar Year 2010)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/kswi-37bp

In [83]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2011__Data_for_Calendar_Year_2010_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2011__Data_for_Calendar_Year_2010_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2011__Data_for_Calendar_Year_2010_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2011__Data_for_Calendar_Year_2010_.head()

(10329, 164)


,Building ID,Facility Name,Period Ending Date,Rating,Address 1,Address 2,City,State,Postcode,County,Total Floor Space (Sq. Ft.),Facility Type,Last Modified Date,Building Profile Status,Eligibility for ENERGY STAR,ENERGY STAR Application Status,Year(s) Labeled,Current Rating (1-100),Target Rating (1-100),Current Direct GHG Emissions (MtCO2e),Current Indirect GHG Emissions (MtCO2e),Current Total GHG Emissions (MtCO2e),Current Energy Period Ending Date,Current Water Period Ending Date,Current Total On-Site Renewable Electric Use (kWh),Percent of Electricity from On-Site Renewable (%),Total Avoided GHG Emissions from Green Power (MtCO2e),Total On-Site Electric Generation (kWh),Total Green Power Purchased (MWh),Current Site Energy Intensity (kBtu/Sq. Ft.),Current Total Site Energy Use (kBtu),National Average Site EUI (kBtu/Sq. Ft.),Target Site Energy Intensity (kBtu/Sq. Ft.),Weather Normalized Site EUI (kBtu/Sq. Ft.),Electricity Use (kBtu),Electricity Use (kWh),Natural Gas Use (kBtu),Natural Gas Use (therms),District Steam Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #56 Use (kBtu),Diesel #2 Use (kBtu),% Difference from National Average Source Energy per Square Foot (%),Current Source Energy Intensity (kBtu/Sq. Ft.),Current Total Source Energy Use (kBtu),Current Weather Normalized Source Energy Intensity (kBtu/Sq. Ft.),National Average Source EUI (kBtu/Sq. Ft.),Bank/Financial Institution- Gross Floor Area (Sq. Ft.),Bank/Financial Institution- Office Air-Conditioned,Bank/Financial Institution- PC Density,Bank/Financial Institution- Weekly operating hours,Bank/Financial Institution- Workers Density,Bank/Financial Institution- Workers on Main Shift,Hospital- Gross Floor Area (Sq. Ft.),"Hospital- Laboratory? (Y=1, N=0)","Hospital- Laundry Facility? (Y=1, N=0)",Hospital- Maximum Number of Floors,Hospital- Number of Buildings,Hospital- Number of Licensed Beds,Hotel- Gross Floor Area (Sq. Ft.),Hotel- Number of Rooms,Hotel- Onsite Laundry (short tons/year),Hotel- Percent Cooled,"Hotel- Presence of cooking facilities? (Y=1, N=0)",House of Worship- Gross Floor Area (Sq. Ft.),House of Worship- PC Density,"House of Worship- Presence of cooking facilities? (Y=1, N=0)",House of Worship- Weekly operating hours,K-12 School- Gross Floor Area (Sq. Ft.),"K-12 School- High School? (Y=1, N=0)","K-12 School- Open Weekends? (Y=1, N=0)",K-12 School- PC Density,K-12 School- Percent Cooled,K-12 School- Walk-in refrig density,"K-12 School- Presence of cooking facilities? (Y=1, N=0)",Medical Office- Gross Floor Area (Sq. Ft.),Medical Office- Number of Workers,Medical Office- Percent Cooled,Medical Office- Weekly operating hours,Multifamily Home- Gross Floor Area (Sq. Ft.),Multifamily Home- Dishwashers in each unit,Multifamily Home- Laundry in common area,Multifamily Home- Laundry in each unit,Multifamily Home- Market Rate or Affordable Housing,Multifamily Home- Number of Floors,Multifamily Home- Number of units,Multifamily Home- Percent Cooled,Multifamily Home- Percent of gross floor area that is common space only,Multifamily Home- Total Number of Bedrooms,Office- Gross Floor Area (Sq. Ft.),Office- Office- Air-Conditioned,Office- PC Density,Office- Weekly operating hours,Office- Workers Density,Other Space Type Name,Other- Gross Floor Area (Sq. Ft.),Other- Number of PCs,Other- Weekly operating hours,Other- Workers on Main Shift,Parking- Enclosed Floor Area (Sq. Ft.),Parking- Gross Floor Area (Sq. Ft.),Parking- Non-Enclosed Floor Area (w/roof) (Sq. Ft.),Parking- Open Floor Area (w/o roof) (Sq. Ft.),Parking- Weekly Hours of Access,Residence Halls/Dormitories- Gross Floor Area (Sq. Ft.),Residence Halls/Dormitories- Percent Cooled,"Residence Halls/Dormitories- Dorm Computer Lab? (Y=1, N=0)","Residence Halls/Dormitories- Dorm Dining Hall? (Y=1, N=0)",Residence Halls/Dormitories- Number of Rooms,Retail- Gross Floor Area (Sq. Ft.),Retail- Cash Register Density,"Retail- Exterior Entrance to the Public? (Y=1, N=0)",Retail- Workers Density,Retail- 

## 31. Energy and Water Data Disclosure for Local Law 84 2016 (Data for Calendar Year 2015)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/77q4-nkfh

In [84]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2016__Data_for_Calendar_Year_2015_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2016__Data_for_Calendar_Year_2015_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2016__Data_for_Calendar_Year_2015_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2016__Data_for_Calendar_Year_2015_.head()

(16139, 256)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Year Ending,"NYC Borough, Block and Lot (BBL)",NYC Building Identification Number (BIN),Address 1,Address 2,City,County,Postcode,Primary Property Type - Self Selected,Primary Property Type - EPA Calculated,National Median Reference Property Type,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,National Median ENERGY STAR Score,ENERGY STAR Certification - Year(s) Certified,ENERGY STAR Certification - Last Approval Date,ENERGY STAR Certification - Eligibility,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),National Median Site EUI (kBtu/ft²),% Difference from National Median Site EUI,Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),National Median Site Energy Use (kBtu),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),% Difference from National Median Source EUI,Source Energy Use (kBtu),Weather Normalized Source Energy Use (kBtu),National Median Source Energy Use (kBtu),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Kerosene Use (kBtu),Propane Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems (kWh),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Green Power - Onsite (kWh),Green Power - Offsite (kWh),Avoided Emissions - Onsite Green Power (Metric Tons CO2e),Avoided Emissions - Offsite Green Power (Metric Tons CO2e),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),National Median Total GHG Emissions (Metric Tons CO2e),Estimated Data Flag - Electricity (Grid Purchase),Estimated Data Flag - Natural Gas,Estimated Data Flag - Fuel Oil (No. 2),Estimated Data Flag - Fuel Oil (No. 4),Estimated Data Flag - Fuel Oil (No. 5 and No. 6),Estimated Data Flag - District Steam,eGRID Output Emissions Rate (kgCO2e/MBtu),Net Emissions (Metric Tons CO2e),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,% Difference from National Median Water/Wastewater Source EUI,CoStar Property ID,LEED US Project ID,Date Property Last Modified,Adult Education - Gross Floor Area (ft²),Ambulatory Surgical Center - Gross Floor Area (ft²),Automobile Dealership - Gross Floor Area (ft²),Bank Branch - Gross Floor Area (ft²),Bank Branch - Number of Computers,Bar/Nightclub - Gross Floor Area (ft²),College/University - Gross Floor Area (ft²),College/University - Number of Computers,Convenience Store with Gas Station - Gross Floor Area (ft²),Convenience Store without Gas Station - Gross Floor Area (ft²),Convention Center - Gross Floor Area (ft²),...,Movie Theater - Gross Floor Area (ft²),Multifamily Housing - Gross Floor Area (ft²),Multifamily Housing - Government Subsidized Housing,Multifamily Housing - Number of Bedrooms,Multifamily Housing - Number of Laundry Hookups in All Units,Multifamily Housing - Number of Laundry Hookups in Common Area(s),Multifamily Housing - Total Number of Residential Living Units,Multif

## 32. Energy and Water Data Disclosure for Local Law 84 2015 (Data for Calendar Year 2014)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/nbun-wekj
    

In [86]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2015__Data_for_Calendar_Year_2014_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2015__Data_for_Calendar_Year_2014_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2015__Data_for_Calendar_Year_2014_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2015__Data_for_Calendar_Year_2014_.head()

(13052, 259)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Year Ending,NYC Building Identification Number (BIN),"NYC Borough, Block and Lot (BBL)",Address 1,Address 2,City,County,Postcode,Primary Property Type - Self Selected,Primary Property Type - EPA Calculated,Property Floor Area (Building(s)) (ft²),National Median Reference Property Type,Year Built,Number of Buildings,Occupancy,Metered Areas (Energy),ENERGY STAR Score,National Median ENERGY STAR Score,ENERGY STAR Certification - Year(s) Certified,ENERGY STAR Certification - Last Approval Date,ENERGY STAR Certification - Next Eligible Date,ENERGY STAR Certification - Eligibility,ENERGY STAR Certification - Application Status,ENERGY STAR Certification - Profile Published,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),National Median Site EUI (kBtu/ft²),Difference from National Median Site EUI,Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),National Median Site Energy Use (kBtu),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),Difference from National Median Source EUI,Source Energy Use (kBtu),Weather Normalized Source Energy Use (kBtu),National Median Source Energy Use (kBtu),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Kerosene Use (kBtu),Propane Use (kBtu),Liquid Propane Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Electricity Use – Generated from Onsite Renewable Systems (kWh),Avoided Emissions - Onsite Green Power (Metric Tons CO2e),Avoided Emissions - Offsite Green Power (Metric Tons CO2e),Estimated Data Flag - Electricity (Grid Purchase),Estimated Data Flag - Natural Gas,Estimated Data Flag - Fuel Oil (No. 2),Estimated Data Flag - Fuel Oil (No. 4),Estimated Data Flag - Fuel Oil (No. 5 and No. 6),Estimated Data Flag - District Steam,Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),eGRID Output Emissions Rate (kgCO2e/MBtu),Net Emissions (Metric Tons CO2e),National Median Total GHG Emissions (Metric Tons CO2e),Green Power - Offsite (kWh),Green Power - Onsite (kWh),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,Municipally Supplied Potable Water: Combined Indoor/Outdoor or Other Cost ($),Difference from National Median Water/Wastewater Source EUI,Energy Cost ($),Energy Cost Intensity ($/ft²),National Median Energy Cost ($),"Investment in Energy Projects, Cumulative ($)","Investment in Energy Projects, Cumulative ($/ft²)","Estimated Savings from Energy Projects, Cumulative ($)","Estimated Savings from Energy Projects, Cumulative ($/ft²)",Electricity (Grid Purchase) Cost ($),Natural Gas Cost ($),Fuel Oil (No. 2) Cost ($),Fuel Oil (No. 4) Cost ($),Fuel Oil (No. 5 and No. 6) Cost ($),District Steam Cost ($),"Water/Wastewater Investment in Energy Projects, Cumulative ($/GPD)","Water/Wastewater Estimated Savings from Energy Projects, Cumulative ($/GPD)",Municipally Supplied Potable Water - Indoor Cost Intensity ($/ft²),...,Medical Office - Number of MRI Machines,Medical Office - Number of Workers on Main Shift,Medical Office - Percent That Can Be Cooled,Medical Office - Percent That Can Be Heated,Medical Office - Weekly Operating Hours,Outpatient Rehabilitation/Physical Therapy - Gross Floor Area (ft²),Urge

## 33. Energy and Water Data Disclosure for Local Law 84 2017 (Data for Calendar Year 2016)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/utpj-74fz

In [87]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2017__Data_for_Calendar_Year_2016_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2017__Data_for_Calendar_Year_2016_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2017__Data_for_Calendar_Year_2016_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2017__Data_for_Calendar_Year_2016_.head()

(16378, 262)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Year Ending,"NYC Borough, Block and Lot (BBL)",NYC Building Identification Number (BIN),Count of Buildings,Address 1,Address 2,City,County,Postcode,Primary Property Type - Self Selected,Primary Property Type - Portfolio Manager-Calculated,National Median Reference Property Type,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,National Median ENERGY STAR Score,ENERGY STAR Certification - Year(s) Certified,ENERGY STAR Certification - Last Approval Date,ENERGY STAR Certification - Eligibility,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),National Median Site EUI (kBtu/ft²),% Difference from National Median Site EUI,Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),Source Energy Use (kBtu),National Median Site Energy Use (kBtu),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),% Difference from National Median Source EUI,Source Energy Use - Adjusted to Current Year (kBtu),Weather Normalized Source Energy Use (kBtu),National Median Source Energy Use (kBtu),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Kerosene Use (kBtu),Propane Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems (kWh),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Green Power - Onsite (kWh),Green Power - Offsite (kWh),Avoided Emissions - Onsite Green Power (Metric Tons CO2e),Avoided Emissions - Offsite Green Power (Metric Tons CO2e),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),National Median Total GHG Emissions (Metric Tons CO2e),Estimated Data Flag - Electricity (Grid Purchase),Estimated Data Flag - Natural Gas,Estimated Data Flag - Fuel Oil (No. 2),Estimated Data Flag - Fuel Oil (No. 4),Estimated Data Flag - Fuel Oil (No. 5 and No. 6),Estimated Data Flag - District Steam,eGRID Output Emissions Rate (kgCO2e/MBtu),Net Emissions (Metric Tons CO2e),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,% Difference from National Median Water/Wastewater Site EUI,CoStar Property ID,LEED US Project ID,Date Property Last Modified,Adult Education - Gross Floor Area (ft²),Ambulatory Surgical Center - Gross Floor Area (ft²),Automobile Dealership - Gross Floor Area (ft²),Bank Branch - Gross Floor Area (ft²),Bank Branch - Number of Computers,College/University - Gross Floor Area (ft²),College/University - Number of Computers,Convenience Store with Gas Station - Gross Floor Area (ft²),Convention Center - Gross Floor Area (ft²),Courthouse - Gross Floor Area (ft²),...,Multifamily Housing - Percent That Can Be Cooled,Multifamily Housing - Percent That Can Be Heated,Multifamily Housing - Resident Population Type,"Multifamily Housing - Number of Bedrooms Density (Number per 1,000 ft²)","Multifamily Housing - Total Number of Residential Living Units Density (Number per 1,000 ft²)",Multifamily Housing - Number of Residential Living Units in a H

## 34. Jobs Created by Energy Cost Savings Program Savings for Businesses - FY2020

Jobs Created by Energy Cost Savings Program Savings for Businesses

https://data.cityofnewyork.us/City-Government/Jobs-Created-by-Energy-Cost-Savings-Program-Saving/yqky-aebb

In [89]:
df_Jobs_Created_by_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2020= \
    pd.read_csv('data/all/Jobs_Created_by_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2020.csv')
print(df_Jobs_Created_by_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2020.shape)
df_Jobs_Created_by_Energy_Cost_Savings_Program_Savings_for_Businesses_FY2020.head()

(58, 17)


,No.,APPLICANT NAME,Address,Postcode,EMP,TOTAL,Savings (Annual),INVESTMENT,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1,"1-10 Bush Terminal Owner, LP dba Industry City(Industry City-683/1) (Bldg. 7&8)","67 34th Street, Brooklyn, NY",11232,25,25,23897.0,"$4,375,000.00",BROOKLYN,40.657359,-74.006423,307.0,38.0,18.0,3336897.0,3.006830e+09,Sunset Park West
1,2,1300 Flushing Owner LLC,"1300 Flushing Avenue, Brooklyn, NY",11237,1,1,NaN,"$9,916,759.00",BROOKLYN,40.709058,-73.922260,304.0,34.0,447.0,3072166.0,3.031690e+09,Bushwick North
2,5,"19-20 Bush Terminal Owner, LP dba Industry City(706/10)(Bldg. 19&20)","168 39th Street, Brooklyn, NY",11232,25,25,24440.0,"$30,000,000.00",BROOKLYN,40.655761,-74.011317,307.0,38.0,18.0,3337105.0,3.007060e+09,Sunset Park West
3,28,"Abel Cine Tech, Inc.","88 35th Street, 4th fl., Brooklyn, NY",11232,86,86,21045.0,NaN,BROOKLYN,40.656475,-74.006484,307.0,38.0,18.0,3336903.0,3.006910e+09,Sunset Park West
4,37,"AECOM Technical Services, Inc. (Ind City)","220 36th Street, Suite 2-5-A, Brooklyn, NY",11232,27,27,9078.0,NaN,BROOKLYN,40.656061,-74.007334,307.0,38.0,2.0,3398687.0,3.006950e+09,Sunset Park West


## 35. Energy and Water Data Disclosure for Local Law 84 2013 (Data for Calendar Year 2012)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/r6ub-zhff
    

In [90]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2013__Data_for_Calendar_Year_2012_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2013__Data_for_Calendar_Year_2012_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2013__Data_for_Calendar_Year_2012_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2013__Data_for_Calendar_Year_2012_.head()

(13382, 201)


,Property Id,Property Name,Year Ending,ENERGY STAR Score,Custom Property ID 1 - ID,Address 1,Address 2,City,State/Province,Postcode,County,Property Floor Area (Buildings and Parking) (ft²),Year Built,Primary Property Type - Self Selected,Date Property Last Modified,Service and Product Provider,ENERGY STAR Certification - Profile Published,Metered Areas (Energy),ENERGY STAR Certification - Eligibility,ENERGY STAR Certification - Application Status,ENERGY STAR Certification - Year(s) Certified,ENERGY STAR Certification - Last Approval Date,Target ENERGY STAR Score,Energy Alerts,Space Alerts,Direct GHG Emissions (MtCO2e),Indirect GHG Emissions (MtCO2e),Total GHG Emissions (MtCO2e),eGRID Output Emissions Rate (kgCO2e/MBtu),Energy Current Date,Water Current Date,Electricity - Onsite Renewable System Electricity Used Onsite/Coincident (kWh),Electricity Sourced from Onsite Renewable Systems (% of Electricity - Grid Purchase and Onsite),Avoided Emissions - Offsite Green Power (MtCO2e),Onsite Renewable System Generation (kWh),Green Power - Offsite (kWh),Onsite Renewable System Electricity Exported (kWh),Site EUI (kBtu/ft²),Site Energy Use (kBtu),National Median Site EUI (kBtu/ft²),Target Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase and Onsite (kWh),Natural Gas Use (kBtu),Natural Gas Use (therms),District Steam Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Other Use (kBtu),District Chilled Water Use (kBtu),District Hot Water Use (kBtu),Estimated Data Flag - Electricity (Grid Purchase),Estimated Data Flag - Electricity (On-Site Solar),Estimated Data Flag - Natural Gas,Estimated Data Flag - District Steam,Estimated Data Flag - Fuel Oil (No. 2),Estimated Data Flag - Fuel Oil (No. 4),Estimated Data Flag - Fuel Oil (No. 5 and No. 6),Estimated Data Flag - Diesel,Estimated Data Flag - Other,Estimated Data Flag - District Hot Water,Estimated Data Flag - District Chilled Water,Percent Better than National Median Site EUI,Source EUI (kBtu/ft²),Source Energy Use (kBtu),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),Percent Better than National Median Source EUI,Bank Branch - Gross Floor Area (ft²),Bank Branch - Percent That Can Be Cooled,"Bank Branch - Computer Density (Number per 1,000 ft²)",Bank Branch - Weekly Operating Hours,"Bank Branch - Worker Density (Number per 1,000 ft²)",Bank Branch - Number of Workers on Main Shift,Hospital (General Medical & Surgical) - Gross Floor Area (ft²),Hospital (General Medical & Surgical) - Laboratory,Hospital (General Medical & Surgical) - Onsite Laundry Facility,Hospital (General Medical & Surgical) - Maximum Number of Floors,Hospital (General Medical & Surgical)- Full Time Equivalent (FTE) Workers,Hospital (General Medical & Surgical) - Number of Staffed Beds,Hospital (General Medical & Surgical) - Number of MRI Machines,Hotel - Gross Floor Area (ft²),Hotel - Full Service Spa Floor Area (ft²),Hotel - Gym/fitness Center Floor Area (ft²),Hotel- Number of Rooms,"Hotel - Room Density (Number per 1,000 ft²)",Hotel - Amount of Laundry Processed On-site Annually (short tons/year),Hotel - Type of Laundry Facility,Hotel - Percent That Can Be Cooled,Hotel - Cooking Facilities,"Hotel - Worker Density (Number per 1,000 ft²)",Worship Facility - Gross Floor Area (ft²),"Worship Facility - Computer Density (Number per 1,000 ft²)",Worship Facility - Weekly Operating Hours,Worship Facility - Cooking Facilities,K-12 School - Gross Floor Area (ft²),K-12 School - High School,...,"K-12 School - Computer Density (Number per 1,000 ft²)",K-12 School - Percent That Can Be Cooled,"K-12 School - Refrigeration Density (Number per 1,000 ft²)",K-12 School - Cooking Facilities,Medical Office - Gross Floor Area (ft²),Medical Office - Number of Workers on Main Shift,Medical Office - Percent That Can Be Cooled,Medical Office - Weekly Operating Hours,Multifamily Housing

## 36. NYC Covered Building's List (2018)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.
    
https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2018-/6pv9-jmn5
        

In [92]:
df_NYC_Covered_Building_s_List__2018_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2018_.csv')
print(df_NYC_Covered_Building_s_List__2018_.shape)
df_NYC_Covered_Building_s_List__2018_.head()

(22718, 20)


,10 Digit BBL,Required to report automated water data from DEP?,Borocode,Block,Lot,Number of Buildings,Tax Class,Building Class,Street Number,Street Name,Postcode,City,DOF Square Footage,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1000047501,N,1,4,7501,1,2.0,R0,1,WATER STREET,10038,Manhattan,2542563,40.702444,-74.012742,101.0,1.0,9.0,1000005.0,Battery Park City-Lower Manhattan
1,1000050010,N,1,5,10,1,4.0,O4,115,BROAD STREET,10004,Manhattan,1016406,40.702486,-74.011357,101.0,1.0,9.0,1000007.0,Battery Park City-Lower Manhattan
2,1000057501,N,1,5,7501,1,2.0,R0,125,BROAD STREET,10004,Manhattan,1354691,40.702112,-74.011249,101.0,1.0,9.0,1000006.0,Battery Park City-Lower Manhattan
3,1000080039,Y,1,8,39,1,4.0,H3,32,PEARL STREET,10004,Manhattan,46724,40.703235,-74.012421,101.0,1.0,9.0,1078968.0,Battery Park City-Lower Manhattan
4,1000080051,N,1,8,51,1,4.0,H2,6,WATER STREET,10004,Manhattan,102407,40.702744,-74.012201,101.0,1.0,9.0,1090472.0,Battery Park City-Lower Manhattan


## 37. NYC Covered Building's List (2016)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.
    
https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2016-/pu45-qynj

In [94]:
df_NYC_Covered_Building_s_List__2016_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2016_.csv')
print(df_NYC_Covered_Building_s_List__2016_.shape)
df_NYC_Covered_Building_s_List__2016_.head()

(12627, 19)


,10 Digit BBL,Automatic Water Benchmarking Eligible,Borocode,Block,Lot,Number of Buildings,Tax Class,Street Number,Street Address,Postcode,Borough,DOF Square Footage*,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1000047501,No,1,4,7501,1,2,1,WATER STREET,10004,Manhattan,2542563,40.702444,-74.012742,101.0,1.0,9.0,1000005.0,Battery Park City-Lower Manhattan
1,1000050010,No,1,5,10,1,4,115,BROAD STREET,10004,Manhattan,1016406,40.702486,-74.011357,101.0,1.0,9.0,1000007.0,Battery Park City-Lower Manhattan
2,1000057501,No,1,5,7501,1,2,125,BROAD STREET,10004,Manhattan,1354691,40.702112,-74.011249,101.0,1.0,9.0,1000006.0,Battery Park City-Lower Manhattan
3,1000080051,No,1,8,51,1,4,6,WATER STREET,10004,Manhattan,103453,40.702744,-74.012201,101.0,1.0,9.0,1090472.0,Battery Park City-Lower Manhattan
4,1000087501,No,1,8,7501,1,2,39,WHITEHALL STREET,10004,Manhattan,169061,40.703037,-74.012969,101.0,1.0,9.0,1087700.0,Battery Park City-Lower Manhattan


## 38. NYC Covered Building's List (2017)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.
    
https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2017-/xirx-msjc

In [95]:
df_NYC_Covered_Building_s_List__2017_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2017_.csv')
print(df_Energy_Demand_Response_Program.shape)
df_NYC_Covered_Building_s_List__2017_.head()

(26, 7)


,10 Digit BBL,"Is this required to comply by May 1st, 2017?",Is this required to report automated water data from DEP?,Borocode,Block,Lot,Number of Buildings,Tax Class,Street Number,Street Address,Postcode,Borough,DOF Square Footage,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1000010010,Yes,No,1,1,10,1,4,1,GOVERNORS ISLAND,10004,Manhattan,2725731,40.690096,-74.012866,101.0,1.0,5.0,1086389.0,park-cemetery-etc-Manhattan
1,1000030001,Yes,No,1,3,1,1,4,10,BATTERY PARK,10004,Manhattan,945425,40.702880,-74.015523,101.0,1.0,319.0,1000000.0,park-cemetery-etc-Manhattan
2,1000047501,Yes,No,1,4,7501,1,2,1,WATER STREET,10004,Manhattan,2542563,40.702444,-74.012742,101.0,1.0,9.0,1000005.0,Battery Park City-Lower Manhattan
3,1000050010,Yes,No,1,5,10,1,4,115,BROAD STREET,10004,Manhattan,1016406,40.702486,-74.011357,101.0,1.0,9.0,1000007.0,Battery Park City-Lower Manhattan
4,1000057501,Yes,No,1,5,7501,1,2,125,BROAD STREET,10004,Manhattan,1354691,40.702112,-74.011249,101.0,1.0,9.0,1000006.0,Battery Park City-Lower Manhattan


## 39. NYC Covered Building's List (2010)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.
    
https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2010-/2vr2-vcvz

In [97]:
df_NYC_Covered_Building_s_List__2010_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2010_.csv')
print(df_NYC_Covered_Building_s_List__2010_.shape)
df_NYC_Covered_Building_s_List__2010_.head()

(92, 7)


,10 DIGIT BBL,BOROUGH,BLOCK,LOT,STREET ADDRESS,ZIP CODE,BUILDING SQUARE\nFOOTAGE
0,1000010010,1,1,10,GOVERNORS ISLAND,10004,2725731
1,1000010101,1,1,101,LIBERTY ISLAND,10004,541886
2,1000010201,1,1,201,ELLIS ISLAND,10004,603130
3,1000030010,1,3,10,SOUTH STREET,10004,123800
4,1000047501,1,4,7501,WATER STREET,10004,2621563


## 40. NYC Covered Building's List (2012)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.
    
https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2012-/63bz-5n3b

In [99]:
df_NYC_Covered_Building_s_List__2012_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2012_.csv')
print(df_NYC_Covered_Building_s_List__2012_.shape)
df_NYC_Covered_Building_s_List__2012_.head()

(13196, 18)


,10 Digit BBL,Borocode,Block,Lot,Number of Buildings,Tax Class,Street Number,Street Address,Borough,Postcode,DOF Square Footage*,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1000010010,1,1,10,1,4,1,GOVERNORS ISLAND,MANHATTAN,10004,2725731,40.690096,-74.012866,101.0,1.0,5.0,1086389.0,park-cemetery-etc-Manhattan
1,1000020002,1,2,2,0,4,NaN,MARGINAL STREET,MANHATTAN,10004,105368,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000047501,1,4,7501,1,2,1,WATER STREET,MANHATTAN,10004,2621563,40.702444,-74.012742,101.0,1.0,9.0,1000005.0,Battery Park City-Lower Manhattan
3,1000057501,1,5,7501,1,2,125,BROAD STREET,MANHATTAN,10004,1354691,40.702112,-74.011249,101.0,1.0,9.0,1000006.0,Battery Park City-Lower Manhattan
4,1000087501,1,8,7501,1,2,39,WHITEHALL STREET,MANHATTAN,10004,169061,40.703037,-74.012969,101.0,1.0,9.0,1087700.0,Battery Park City-Lower Manhattan


## 41. Local Law 84 2021 (Monthly Data for Calendar Year 2020)

Monthly whole building electricity and natural gas consumption data in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Local-Law-84-2021-Monthly-Data-for-Calendar-Year-2/in83-58q5

In [101]:
df_Local_Law_84_2021__Monthly_Data_for_Calendar_Year_2020_= \
    pd.read_csv('data/all/Local_Law_84_2021__Monthly_Data_for_Calendar_Year_2020_.csv')
print(df_Local_Law_84_2021__Monthly_Data_for_Calendar_Year_2020_.shape)
df_Local_Law_84_2021__Monthly_Data_for_Calendar_Year_2020_.head()

(334044, 7)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Month,Electricity Use (kBtu),Natural Gas Use (kBtu)
0,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Jan-20,2175731.1,Not Available
1,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Feb-20,1902208.9,Not Available
2,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Mar-20,1847793.7,Not Available
3,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Apr-20,1613573.0,Not Available
4,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,May-20,1747078.0,Not Available


## 42. NYC Covered Building's List (2011)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.

https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2011-/wf9c-gupx

In [103]:
df_NYC_Covered_Building_s_List__2011_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2011_.csv')
print(df_NYC_Covered_Building_s_List__2011_.shape)
df_NYC_Covered_Building_s_List__2011_.head()

(13258, 16)


,10 Digit BBL,Borocode,Block,Lot,StreetNumber,Street Address,Postcode,DOF Square Footage*,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1015317501,1,1531,7501,205,EAST 85 STREET,10028,303383,MANHATTAN,40.777977,-73.954079,108.0,5.0,14601.0,1087920.0,Yorkville
1,1015320001,1,1532,1,203,EAST 86 STREET,10028,256810,MANHATTAN,40.778674,-73.953613,108.0,5.0,14602.0,1048787.0,Yorkville
2,1015320004,1,1532,4,1533,3 AVENUE,10028,222000,MANHATTAN,40.779061,-73.953797,108.0,5.0,14602.0,1077854.0,Yorkville
3,1015320015,1,1532,15,1740,BROADWAY,10028,57745,MANHATTAN,40.765098,-73.982095,105.0,4.0,137.0,1024877.0,Midtown-Midtown South
4,1015320016,1,1532,16,230,E 85TH ST,10028,104440,MANHATTAN,40.777787,-73.953672,108.0,5.0,14601.0,1048745.0,Yorkville


## 43. NYC Covered Building's List (2014)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.
    
    
https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2014-/c4p6-rvjs

In [105]:
df_NYC_Covered_Building_s_List__2014_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2014_.csv')
print(df_NYC_Covered_Building_s_List__2014_.shape)
df_NYC_Covered_Building_s_List__2014_.head()

(13678, 19)


,10 Digit BBL,Automatic Water Benchmarking Eligible,Borocode,Block,Lot,Number of Buildings,Tax Class,Street Number,Street Address,Postcode,Borough,DOF Square Footage*,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1000010010,No,1,1,10,1,4,1,GOVERNORS ISLAND,10004,MANHATTAN,2725731,40.690096,-74.012866,101.0,1.0,5.0,1086389.0,park-cemetery-etc-Manhattan
1,1000030001,No,1,3,1,1,4,10,BATTERY PARK,10004,MANHATTAN,945425,40.702880,-74.015523,101.0,1.0,319.0,1000000.0,park-cemetery-etc-Manhattan
2,1000047501,No,1,4,7501,1,2,1,WATER STREET,10004,MANHATTAN,2542563,40.702444,-74.012742,101.0,1.0,9.0,1000005.0,Battery Park City-Lower Manhattan
3,1000050010,No,1,5,10,1,4,115,BROAD STREET,10004,MANHATTAN,1016406,40.702486,-74.011357,101.0,1.0,9.0,1000007.0,Battery Park City-Lower Manhattan
4,1000057501,No,1,5,7501,1,2,125,BROAD STREET,10004,MANHATTAN,1354691,40.702112,-74.011249,101.0,1.0,9.0,1000006.0,Battery Park City-Lower Manhattan


## 44. NYC Covered Building's List (2013)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.
    
https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2013-/wg5a-mubp

In [107]:
df_NYC_Covered_Building_s_List__2013_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2013_.csv')
print(df_NYC_Covered_Building_s_List__2013_.shape)
df_NYC_Covered_Building_s_List__2013_.head()

(13448, 15)


,BBL_10,STREET_NO_PROPERTY,STREET_1_PROPERTY,STREET_2_PROPERTY,CITY_PROPERTY,Postcode,ADDR_STATE_PROPERTY,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,5035857501,100,COLFAX AVENUE,NaN,STATEN ISLAND,10306,NY,STATEN IS,40.579906,-74.107779,502.0,50.0,11402.0,5095450.0,New Dorp-Midland Beach
1,5038510001,NaN,FR CAPODANNO BLVD,NaN,STATEN ISLAND,0,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5039010017,2424,HYLAN BOULEVARD,NaN,STATEN ISLAND,10306,NY,STATEN IS,40.571986,-74.108220,502.0,50.0,11202.0,5055347.0,New Dorp-Midland Beach
3,5039690001,2660,HYLAN BOULEVARD,NaN,STATEN ISLAND,10306,NY,STATEN IS,40.566994,-74.113237,502.0,50.0,12804.0,NaN,New Dorp-Midland Beach
4,5039690031,107,MILL ROAD,NaN,STATEN ISLAND,10306,NY,STATEN IS,40.566291,-74.108585,502.0,50.0,12804.0,5000000.0,New Dorp-Midland Beach


## 45. NYC Covered Building's List (2015)

The covered building list identifies properties required to comply with Local Law 84: Benchmarking.

https://data.cityofnewyork.us/Environment/NYC-Covered-Building-s-List-2015-/4hbx-ab3c
    

In [109]:
df_NYC_Covered_Building_s_List__2015_= \
    pd.read_csv('data/all/NYC_Covered_Building_s_List__2015_.csv')
print(df_NYC_Covered_Building_s_List__2015_.shape)
df_NYC_Covered_Building_s_List__2015_.head()

(13678, 19)


,10 Digit BBL,Automatic Water Benchmarking Eligible,Borocode,Block,Lot,Number of Buildings,Tax Class,Street Number,Street Address,Postcode,Borough,DOF Square Footage*,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA
0,1000010010,No,1,1,10,1,4,1,GOVERNORS ISLAND,10004,MANHATTAN,2725731,40.690096,-74.012866,101.0,1.0,5.0,1086389.0,park-cemetery-etc-Manhattan
1,1000030001,No,1,3,1,1,4,10,BATTERY PARK,10004,MANHATTAN,945425,40.702880,-74.015523,101.0,1.0,319.0,1000000.0,park-cemetery-etc-Manhattan
2,1000047501,No,1,4,7501,1,2,1,WATER STREET,10004,MANHATTAN,2542563,40.702444,-74.012742,101.0,1.0,9.0,1000005.0,Battery Park City-Lower Manhattan
3,1000050010,No,1,5,10,1,4,115,BROAD STREET,10004,MANHATTAN,1016406,40.702486,-74.011357,101.0,1.0,9.0,1000007.0,Battery Park City-Lower Manhattan
4,1000057501,No,1,5,7501,1,2,125,BROAD STREET,10004,MANHATTAN,1354691,40.702112,-74.011249,101.0,1.0,9.0,1000006.0,Battery Park City-Lower Manhattan


## 46. DCLA Cultural Institutions Group Funding

Operating and energy support funding provided to members of the cultural institution group (CIG).

https://data.cityofnewyork.us/City-Government/DCLA-Cultural-Institutions-Group-Funding/ka27-qx5k

In [111]:
df_DCLA_Cultural_Institutions_Group_Funding= \
    pd.read_csv('data/all/DCLA_Cultural_Institutions_Group_Funding.csv')
print(df_DCLA_Cultural_Institutions_Group_Funding.shape)
df_DCLA_Cultural_Institutions_Group_Funding.head()

(239, 4)


,Cultural Institutions Group (CIG),Fiscal Year (FY),Operating Support,Energy Support
0,American Museum of Natural History,2014.0,8483800.0,7611998.0
1,Bronx County Historical Society,2014.0,155178.0,16172.0
2,Bronx Museum of the Arts,2014.0,579384.0,212629.0
3,Brooklyn Academy of Music,2014.0,2006587.0,864833.0
4,Brooklyn Botanic Garden,2014.0,3404959.0,572883.0


## 47. DOB Sustainability Compliance Map: Local Law 33


Data used to feed Sustainability Compliance Map. This data comes from the NYC Department of Finance, the US EPA's Portfolio Manager, and grading metric based on Local Law 95 of 2019.

https://data.cityofnewyork.us/Business/DOB-Sustainability-Compliance-Map-Local-Law-33/355w-xvp2

In [112]:
df_DOB_Sustainability_Compliance_Map__Local_Law_33= \
    pd.read_csv('data/all/DOB_Sustainability_Compliance_Map__Local_Law_33.csv')
print(df_DOB_Sustainability_Compliance_Map__Local_Law_33.shape)
df_DOB_Sustainability_Compliance_Map__Local_Law_33.head()

(21681, 11)


,Block,Lot,Building_Class,Tax_Class,Building_Count,DOF_Gross_Square_Footage,Address,BoroughName,BBL,ENERGY STAR Score,LetterScore
0,1,10,Y4,0,124,2598091,920 GRESHAM ROAD,MANHATTAN,1000010010,1,D
1,2,23,T2,0,1,24346,20 SOUTH STREET,MANHATTAN,1000020023,0,F
2,4,7501,R0,2,1,2542563,1 WATER STREET,MANHATTAN,1000047501,61,C
3,5,7501,R0,2,1,1354691,125 BROAD STREET,MANHATTAN,1000057501,76,B
4,8,7501,R0,2,1,169061,2 WATER STREET,MANHATTAN,1000087501,13,D


## 48. Energy and Water Data Disclosure for Local Law 84 2014 (Data for Calendar Year 2013)

Data and metrics on water and energy consumption in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/yr5p-wjer

In [120]:
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2014__Data_for_Calendar_Year_2013_= \
    pd.read_csv('data/all/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2014__Data_for_Calendar_Year_2013_.csv')
print(df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2014__Data_for_Calendar_Year_2013_.shape)
df_Energy_and_Water_Data_Disclosure_for_Local_Law_84_2014__Data_for_Calendar_Year_2013_.head()

(13133, 255)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Year Ending,NYC Building Identification Number (BIN),"NYC Borough, Block and Lot (BBL)",Custom Property ID 1 - Name,Custom Property ID 1 - ID,Address 1,Address 2,City,County,Postcode,Primary Property Type - Self Selected,Primary Property Type - EPA Calculated,Property Floor Area (Building(s)) (ft²),National Median Reference Property Type,Year Built,Number of Buildings,Occupancy,Metered Areas (Energy),ENERGY STAR Score,National Median ENERGY STAR Score,ENERGY STAR Certification - Year(s) Certified,ENERGY STAR Certification - Last Approval Date,ENERGY STAR Certification - Next Eligible Date,ENERGY STAR Certification - Eligibility,ENERGY STAR Certification - Application Status,ENERGY STAR Certification - Profile Published,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),National Median Site Energy Use (kBtu),National Median Site EUI (kBtu/ft²),Percent Better than National Median Site EUI,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Source Energy Use (kBtu),National Median Source Energy Use (kBtu),National Median Source EUI (kBtu/ft²),Percent Better than National Median Source EUI,Natural Gas Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),District Steam Use (kBtu),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kWh),Weather Normalized Site Natural Gas Use (therms),Weather Normalized Site Electricity (kWh),Energy Alerts,Property Use Detail Alerts,Estimated Data Flag - Electricity (Grid Purchase),Estimated Data Flag - Natural Gas,Estimated Data Flag - Fuel Oil (No. 2),Estimated Data Flag - Fuel Oil (No. 4),Estimated Data Flag - Fuel Oil (No. 5 and No. 6),Estimated Data Flag - District Steam,Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),eGRID Output Emissions Rate (kgCO2e/MBtu),Net Emissions (Metric Tons CO2e),National Median Total GHG Emissions (Metric Tons CO2e),Green Power - Offsite (kWh),Green Power - Onsite (kWh),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,Municipally Supplied Potable Water: Combined Indoor/Outdoor or Other Use (kgal),Municipally Supplied Potable Water - Indoor Intensity (gal/ft²),Other Water Sources: Combined Indoor/Outdoor or Other Use (kgal),Water Alerts,Percent Better than National Median Water/Wastewater Source EUI,Weather Normalized Source Energy Use (kBtu),Adult Education - Gross Floor Area (ft²),Ambulatory Surgical Center - Gross Floor Area (ft²),Aquarium - Gross Floor Area (ft²),Automobile Dealership - Gross Floor Area (ft²),Bank Branch - Gross Floor Area (ft²),Bank Branch - Number of Computers,Bar/Nightclub - Gross Floor Area (ft²),Bowling Alley - Gross Floor Area (ft²),Casino - Gross Floor Area (ft²),College/University - Gross Floor Area (ft²),College/University - Number of Computers,Convenience Store with Gas Station - Gross Floor Area (ft²),Convenience Store without Gas Station - Gross Floor Area (ft²),Convention Center - Gross Floor Area (ft²),Courthouse - Gross Floor Area (ft²),Data Center - PDU Input Site Energy (kWh),Data Center - PDU Output Site Energy (kWh),Data Center - IT Equipment Input Site Energy (kWh),Data Center - IT Site Energy (kWh),Data Center - IT Source Energy (kBtu),...,Multifamily Housing - Gross Floor Area (ft²),Multifamily Housing - Maximum Number of Floors,Multifamily Housing - Number of Bedrooms,Multifamily Housing - Number of Dishwasher Hookups,Multifamily Housing - Number of Dishwasher Hookups in All Units,Multifam

## 49. Local Law 84 2020 (Monthly Data for Calendar Year 2019)

Monthly whole building electricity and natural gas consumption data in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

https://data.cityofnewyork.us/Environment/Local-Law-84-2020-Monthly-Data-for-Calendar-Year-2/evu4-6zyr

In [122]:
df_Local_Law_84_2020__Monthly_Data_for_Calendar_Year_2019_= \
    pd.read_csv('data/all/Local_Law_84_2020__Monthly_Data_for_Calendar_Year_2019_.csv')
print(df_Local_Law_84_2020__Monthly_Data_for_Calendar_Year_2019_.shape)
df_Local_Law_84_2020__Monthly_Data_for_Calendar_Year_2019_.head()

(335616, 7)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Month,Natural Gas Use (kBtu),Electricity Use (kBtu)
0,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Jan-19,Not Available,2036389.3
1,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Feb-19,Not Available,1935369.4
2,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Mar-19,Not Available,2137344.7
3,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Apr-19,Not Available,1979256.0
4,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,May-19,Not Available,2058142.7


## 50. Local Law 84 2019 (Monthly Data for Calendar Year 2018)

Monthly whole building electricity and natural gas consumption data in privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2.

To understand more about full building characteristics and annual metrics, visit this link to view the annual water and energy consumption data set. Each property in these two data sets can be linked using the “Property ID” column, which serves as a key.


https://data.cityofnewyork.us/Environment/Local-Law-84-2019-Monthly-Data-for-Calendar-Year-2/njuk-taxk
    
    

In [124]:
df_Local_Law_84_2019__Monthly_Data_for_Calendar_Year_2018_= \
    pd.read_csv('data/all/Local_Law_84_2019__Monthly_Data_for_Calendar_Year_2018_.csv')
print(df_Local_Law_84_2019__Monthly_Data_for_Calendar_Year_2018_.shape)
df_Local_Law_84_2019__Monthly_Data_for_Calendar_Year_2018_.head()

(309528, 7)


,Property Id,Property Name,Parent Property Id,Parent Property Name,Month,Natural Gas Use (kBtu),Electricity Use (kBtu)
0,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Jan-18,Not Available,2191816.3
1,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Feb-18,Not Available,1881934.0
2,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Mar-18,Not Available,1960262.3
3,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Apr-18,Not Available,1842159.3
4,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,May-18,Not Available,1893434.5


## 51. Heating Gas Consumption And Cost (2010 - Feb 2022)

Monthly consumption and cost data by borough and development. Data set includes utility vendor and meter information.

https://data.cityofnewyork.us/Housing-Development/Heating-Gas-Consumption-And-Cost-2010-Feb-2022-/it56-eyq4

In [127]:
df_Heating_Gas_Consumption_And_Cost__2010__Feb_2022_= \
    pd.read_csv('data/all/Heating_Gas_Consumption_And_Cost__2010_-__Feb_2022_.csv')
print(df_Heating_Gas_Consumption_And_Cost__2010__Feb_2022_.shape)
df_Heating_Gas_Consumption_And_Cost__2010__Feb_2022_.head()

(214635, 25)


,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,AMP #,Vendor Name,UMIS BILL ID,Revenue Month,Service Start Date,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (Therms),ES Commodity,Underlying Utility
0,BETANCES I,BRONX,BETANCES I,NaN,NONE,NaN,211.0,326,B021100,FEDERAL,NY005012110P,CONSOLIDATED EDISON COMPANY OF NY,1841847,2010-01,12/09/2009,01/31/2010,53.0,2545269,N,0.00,NaN,Yes,0.00,UTILITY GAS,ConEd
1,ADAMS,BRONX,ADAMS,BLD 04,NONE,BLD 1-7,118.0,248,B011800,FEDERAL,NY005001180P,CONSOLIDATED EDISON COMPANY OF NY,1838631,2010-01,12/24/2009,01/26/2010,33.0,3299599,N,78292.97,Trans Res MultiDwell Heat Dual,Yes,136632.00,UTILITY GAS,ConEd
2,ADAMS,BRONX,ADAMS,BLD 04,Not Applicable,NaN,118.0,248,B011800,FEDERAL,NY005001180P,HESS CORPORATION (905243) - 2,1851664,2010-01,12/25/2009,01/26/2010,32.0,3.95416E+14,N,133779.54,NaN,Yes,136632.00,BROKERED GAS,ConEd
3,ALBANY/ALBANY II,BROOKLYN,ALBANY/ALBANY II,ALBANY BLD 04,AMR,BLD 1-6,31.0,524,K003100,FEDERAL,NY005010310P,National Grid NYC,2215468,2010-01,12/29/2009,01/28/2010,30.0,08372362-66.8%,N,44335.21,T3 Tran Mult Fam,Yes,153899.18,UTILITY GAS,NatGrid NYC
4,ALBANY/ALBANY II,BROOKLYN,ALBANY/ALBANY II,ALBANY BLD 04,AMR,BLD 7-9,85.0,524,K008500,FEDERAL,NY005010310P,National Grid NYC,2215468,2010-01,12/29/2009,01/28/2010,30.0,08372362-33.2%,N,22034.86,T3 Tran Mult Fam,Yes,76488.82,UTILITY GAS,NatGrid NYC


## 52. Cooking Gas Consumption And Cost (2010 - Feb 2022)

Monthly consumption and cost data by borough and development. Data set includes utility vendor and meter information.

https://data.cityofnewyork.us/Housing-Development/Cooking-Gas-Consumption-And-Cost-2010-Feb-2022-/avhb-5jhc

In [130]:
df_Cooking_Gas_Consumption_And_Cost__2010_Feb_2022_= \
    pd.read_csv('data/all/Cooking_Gas_Consumption_And_Cost__2010_-_Feb_2022_.csv')
print(df_Cooking_Gas_Consumption_And_Cost__2010_Feb_2022_.shape)
df_Cooking_Gas_Consumption_And_Cost__2010_Feb_2022_.head()

(585050, 25)


,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,AMP #,Vendor Name,UMIS BILL ID,Revenue Month,Service Start Date,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (Therms),ES Commodity,Underlying Utility
0,ALBANY II,BROOKLYN,ALBANY/ALBANY II,ALBANY II BLD 09,AMR,NaN,85,524,K008500,FEDERAL,NY005010310P,National Grid NYC,8566316,2020-01-01T00:00:00.000,12/26/2019,01/27/2020,32.0,904846.0,N,295.16,T1A TRAN RES N-Ht,Yes,399.45,UTILITY GAS,NatGrid NYC
1,ALBANY II,BROOKLYN,ALBANY/ALBANY II,ALBANY II BLD 09,AMR,NaN,85,524,K008500,FEDERAL,NY005010310P,National Grid NYC,8566316,2020-01-01T00:00:00.000,12/26/2019,01/27/2020,32.0,924103.0,N,71.11,T1A TRAN RES N-Ht,Yes,96.24,UTILITY GAS,NatGrid NYC
2,ALBANY,BROOKLYN,ALBANY/ALBANY II,ALBANY II BLD 07,AMR,NaN,31,524,K003100,FEDERAL,NY005010310P,National Grid NYC,8664879,2020-02-01T00:00:00.000,01/27/2020,02/25/2020,29.0,152644.0,N,276.07,T1A TRAN RES N-Ht,Yes,371.88,UTILITY GAS,NatGrid NYC
3,ALBANY,BROOKLYN,ALBANY/ALBANY II,ALBANY BLD 03,AMR,NaN,31,524,K003100,FEDERAL,NY005010310P,National Grid NYC,8664879,2020-02-01T00:00:00.000,01/27/2020,02/25/2020,29.0,379350.0,N,0.00,T1A TRAN RES N-Ht,Yes,0.00,UTILITY GAS,NatGrid NYC
4,ALBANY,BROOKLYN,ALBANY/ALBANY II,ALBANY BLD 04,AMR,NaN,31,524,K003100,FEDERAL,NY005010310P,National Grid NYC,8664879,2020-02-01T00:00:00.000,01/27/2020,02/25/2020,29.0,463027.0,N,93.82,T1A TRAN RES N-Ht,Yes,126.38,UTILITY GAS,NatGrid NYC


## 53. DOB Sustainability Compliance Map

This data comes from the NYC Department of Finance, the US EPA's Portfolio Manager, and grading metric based on Local Law 95 of 2019.

https://data.cityofnewyork.us/dataset/DOB-Sustainability-Compliance-Map/4hxk-b29t


In [132]:
df_LL33= \
    pd.read_csv('data/all/LL33.csv')
print(df_LL33.shape)
df_LL33.head()

(21681, 11)


,Block,Lot,Building_Class,Tax_Class,Building_Count,DOF_Gross_Square_Footage,Address,BoroughName,BBL,ENERGY STAR Score,LetterScore
0,1,10,Y4,0,124,2598091,920 GRESHAM ROAD,MANHATTAN,1000010010,1,D
1,2,23,T2,0,1,24346,20 SOUTH STREET,MANHATTAN,1000020023,0,F
2,4,7501,R0,2,1,2542563,1 WATER STREET,MANHATTAN,1000047501,61,C
3,5,7501,R0,2,1,1354691,125 BROAD STREET,MANHATTAN,1000057501,76,B
4,8,7501,R0,2,1,169061,2 WATER STREET,MANHATTAN,1000087501,13,D


In [133]:
df_LL97= \
    pd.read_csv('data/all/LL97.csv')
print(df_LL97.shape)
df_LL97.head()

(12667, 9)


,Block,Lot,TaxClass,BldgClass,GBA,Building_Count,Address,BBL,Borough
0,1,101,4.0,P7,541886,10,1 LIBERTY ISLAND,1000010101,Manhattan
1,1,201,4.0,Z9,603130,6,1 ELLIS ISLAND,1000010201,Manhattan
2,4,7501,2.0,R0,2542563,1,1 WATER STREET,1000047501,Manhattan
3,5,7501,2.0,R0,1354691,1,125 BROAD STREET,1000057501,Manhattan
4,8,7501,2.0,R0,169061,1,2 WATER STREET,1000087501,Manhattan


## 54. Electric Consumption And Cost (2010 - Feb 2022)

Monthly consumption and cost data by borough and development. Data set includes utility vendor and meter information.

https://data.cityofnewyork.us/Housing-Development/Electric-Consumption-And-Cost-2010-Feb-2022-/jr24-e7cr

In [135]:
df_Electric_Consumption_And_Cost__2010_Feb_2022_= \
    pd.read_csv('data/all/Electric_Consumption_And_Cost__2010_-_Feb_2022_.csv')
print(df_Electric_Consumption_And_Cost__2010_Feb_2022_.shape)
df_Electric_Consumption_And_Cost__2010_Feb_2022_.head()

(407031, 27)


,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,AMP #,Vendor Name,UMIS BILL ID,Revenue Month,Service Start Date,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,NY005001180P,NEW YORK POWER AUTHORITY,1842037,2010-01,12/24/2009,01/26/2010,33.0,7223256,N,15396.82,GOV/NYC/068,Yes,128800,7387.97,216.0,2808.0,5200.85
1,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,NY005001180P,NEW YORK POWER AUTHORITY,1844205,2010-02,01/26/2010,02/25/2010,30.0,7223256,N,14556.34,GOV/NYC/068,Yes,115200,6607.87,224.0,2912.0,5036.47
2,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,NY005001180P,NEW YORK POWER AUTHORITY,1841378,2010-03,02/25/2010,03/26/2010,29.0,7223256,N,13904.98,GOV/NYC/068,Yes,103200,5919.55,216.0,2808.0,5177.43
3,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,NY005001180P,NEW YORK POWER AUTHORITY,1847148,2010-04,03/26/2010,04/26/2010,31.0,7223256,N,14764.04,GOV/NYC/068,Yes,105600,6057.22,208.0,2704.0,6002.82
4,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,NY005001180P,NEW YORK POWER AUTHORITY,1848672,2010-05,04/26/2010,05/24/2010,28.0,7223256,N,13729.54,GOV/NYC/068,Yes,97600,5598.34,216.0,2808.0,5323.20


## 55. Heating Oil Consumption And Cost (2010 - Feb 2022)

Monthly consumption and cost data by borough and development. Data set includes utility vendor and meter information.

https://data.cityofnewyork.us/Housing-Development/Heating-Oil-Consumption-And-Cost-2010-Feb-2022-/bhwu-wuzu

In [138]:
df_Heating_Oil_Consumption_And_Cost__2010_Feb_2022_= \
    pd.read_csv('data/all/Heating_Oil_Consumption_And_Cost__2010_-_Feb_2022_.csv')
print(df_Heating_Oil_Consumption_And_Cost__2010_Feb_2022_.shape)
df_Heating_Oil_Consumption_And_Cost__2010_Feb_2022_.head()

(14628, 21)


,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,AMP #,Vendor Name,UMIS BILL ID,Revenue Month,Service Start Date,Service End Date,# days,Meter Number,Estimated,Current Charges,Consumption (GAL)
0,ADAMS,BRONX,ADAMS,NaN,Not Applicable,NaN,118,248,B011800,FEDERAL,NY005001180P,"S.J. Fuel Co., Inc.",2897085,2010-01,12/24/2009,01/26/2010,33,395416152700002-OIL,N,303.47,148.0
1,ADAMS,BRONX,ADAMS,NaN,Not Applicable,NaN,118,248,B011800,FEDERAL,NY005001180P,"S.J. Fuel Co., Inc.",2897086,2010-02,01/26/2010,02/25/2010,30,395416152700002-OIL,N,504.41,246.0
2,ADAMS,BRONX,ADAMS,NaN,Not Applicable,NaN,118,248,B011800,FEDERAL,NY005001180P,"S.J. Fuel Co., Inc.",2897087,2010-03,02/25/2010,03/26/2010,29,395416152700002-OIL,N,200.94,98.0
3,ADAMS,BRONX,ADAMS,NaN,Not Applicable,NaN,118,248,B011800,FEDERAL,NY005001180P,"S.J. Fuel Co., Inc.",2897088,2010-05,04/26/2010,05/24/2010,28,395416152700002-OIL,N,789.42,385.0
4,ADAMS,BRONX,ADAMS,NaN,Not Applicable,NaN,118,248,B011800,FEDERAL,NY005001180P,"S.J. Fuel Co., Inc.",2897089,2010-06,05/24/2010,06/23/2010,30,395416152700002-OIL,N,488.01,238.0


## 56. Pre-Qualified List

The Office of Citywide Procurement established a list of pre-qualified general contractors for furnishing all labor, materials and equipment, necessary and required to perform energy efficient work on various capital and expense funded projects throughout the City.

https://data.cityofnewyork.us/City-Government/Pre-Qualified-List/5hsa-dfq5

In [140]:
df_Pre_Qualified_List= \
    pd.read_csv('data/all/Pre-Qualified_List.csv')
print(df_Pre_Qualified_List.shape)
df_Pre_Qualified_List.head()

(285, 17)


,COMPANY NAME,STREET NUMBER,STREET NAME,CITY,STATE,ZIP CODE,UNIT,Borough Name,Borough Code,Community District,CENSUS TRACT,NTA - NEIGHBORHOOD TABULATION AREA,NTA Name,LATITUDE,LONGITUDE,BBL - BOROUGH BLOCK & LOT,BIN - BUILDING IDENTIFICATION NUMBER
0,A.T.J. Electrical Co. Inc.,122-11,15th Ave.,College Point,NY,11356,NaN,QUEENS,4.0,407.0,929.0,QN23,College Point,40.784234,-73.845604,4.040850e+09,4098344.0
1,AECOM,125,Broad Street,New York,NY,10004,15th Floor,MANHATTAN,1.0,101.0,9.0,MN25,Battery Park City-Lower Manhattan,40.702112,-74.011249,1.000058e+09,1000006.0
2,AECOM,125,Broad Street,New York,NY,10004,15th Floor,MANHATTAN,1.0,101.0,9.0,MN25,Battery Park City-Lower Manhattan,40.702112,-74.011249,1.000058e+09,1000006.0
3,AECOM,125,"Broad Street,",New York,NY,10004,15th Floor,MANHATTAN,1.0,101.0,9.0,MN25,Battery Park City-Lower Manhattan,40.702112,-74.011249,1.000058e+09,1000006.0
4,AECOM,125,Broad Street,New york,NY,10004,15th Floor,MANHATTAN,1.0,101.0,9.0,MN25,Battery Park City-Lower Manhattan,40.702112,-74.011249,1.000058e+09,1000006.0


## 57. Water Consumption And Cost (2013 - Feb 2022)

Monthly consumption and cost data by borough and development. Data set includes utility vendor and meter information.

https://data.cityofnewyork.us/Housing-Development/Water-Consumption-And-Cost-2013-Feb-2022-/66be-66yr

In [141]:
df_Water_Consumption_And_Cost__2013_Feb_2022_= \
    pd.read_csv('data/all/Water_Consumption_And_Cost__2013_-_Feb_2022_.csv')
print(df_Water_Consumption_And_Cost__2013_Feb_2022_.shape)
df_Water_Consumption_And_Cost__2013_Feb_2022_.head()

(46747, 25)


,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,AMP #,Vendor Name,UMIS BILL ID,Revenue Month,Service Start Date,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (HCF),Water&Sewer Charges,Other Charges
0,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,NY005010910P,NEW YORK CITY WATER BOARD,8562430,2020-01,12/23/2019,01/26/2020,34.0,K13060723,N,196.35,Basic Water and Sewer,Yes,19,196.35,0.0
1,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,NY005010910P,NEW YORK CITY WATER BOARD,8667039,2020-02,01/26/2020,02/24/2020,29.0,K13060723,N,258.35,Basic Water and Sewer,Yes,25,258.35,0.0
2,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,NY005010910P,NEW YORK CITY WATER BOARD,8759719,2020-03,02/24/2020,03/23/2020,28.0,K13060723,N,217.02,Basic Water and Sewer,Yes,21,217.02,0.0
3,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,NY005010910P,NEW YORK CITY WATER BOARD,8870760,2020-04,03/23/2020,04/23/2020,31.0,K13060723,N,103.34,Basic Water and Sewer,Yes,10,103.34,0.0
4,BAY VIEW,BROOKLYN,BAY VIEW,BLD 25 - Community Center,NONE,Community Center,92.0,670,K209200,MIXED FINANCE/LLC1,NY005020920P,NEW YORK CITY WATER BOARD,8560969,2020-01,12/23/2019,01/26/2020,34.0,E17250205,N,72.34,Basic Water and Sewer,Yes,7,72.34,0.0


## 58. Steam Consumption And Cost (2010 - Feb 2022)

Monthly consumption and cost data by borough and development. Data set includes utility vendor and meter information.

https://data.cityofnewyork.us/Housing-Development/Steam-Consumption-And-Cost-2010-Feb-2022-/smdw-73pj

In [143]:
df_Steam_Consumption_And_Cost__2010__Feb_2022_= \
    pd.read_csv('data/all/Steam_Consumption_And_Cost__2010_-__Feb_2022_.csv')
print(df_Steam_Consumption_And_Cost__2010__Feb_2022_.shape)
df_Steam_Consumption_And_Cost__2010__Feb_2022_.head()

(4949, 23)


,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,AMP #,Vendor Name,UMIS BILL ID,Revenue Month,Service Start Date,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (Mlbs)
0,AMSTERDAM ADDITION,MANHATTAN,AMSTERDAM ADDITION,BLD 01,AMR,NaN,187,453,M018700,MIXED FINANCE/LLC1,NY005021870P,CONSOLIDATED EDISON COMPANY OF NY,1831828,2010-01,12/21/2009,01/21/2010,31,321060,N,50213.25,SC3,Yes,1612.64
1,AMSTERDAM ADDITION,MANHATTAN,AMSTERDAM ADDITION,BLD 01,AMR,NaN,187,453,M018700,MIXED FINANCE/LLC1,NY005021870P,CONSOLIDATED EDISON COMPANY OF NY,1831828,2010-01,12/21/2009,01/21/2010,31,331048,N,21201.70,SC3,Yes,680.91
2,AMSTERDAM ADDITION,MANHATTAN,AMSTERDAM ADDITION,BLD 01,AMR,NaN,187,453,M018700,MIXED FINANCE/LLC1,NY005021870P,CONSOLIDATED EDISON COMPANY OF NY,1831865,2010-02,01/21/2010,02/19/2010,29,321060,N,22119.58,SC3,Yes,706.22
3,AMSTERDAM ADDITION,MANHATTAN,AMSTERDAM ADDITION,BLD 01,AMR,NaN,187,453,M018700,MIXED FINANCE/LLC1,NY005021870P,CONSOLIDATED EDISON COMPANY OF NY,1831865,2010-02,01/21/2010,02/19/2010,29,331048,N,40960.48,SC3,Yes,1307.76
4,AMSTERDAM ADDITION,MANHATTAN,AMSTERDAM ADDITION,BLD 01,AMR,NaN,187,453,M018700,MIXED FINANCE/LLC1,NY005021870P,CONSOLIDATED EDISON COMPANY OF NY,1831915,2010-03,02/19/2010,03/23/2010,32,321060,N,29303.60,SC3,Yes,947.68


## 59. NYC Clean Heat Dataset (Historical)

NYC Clean Heat dataset

https://data.cityofnewyork.us/City-Government/NYC-Clean-Heat-Dataset-Historical-/8isn-pgv3

In [146]:
df_NYC_Clean_Heat_Dataset__Historical_= \
    pd.read_csv('data/all/NYC_Clean_Heat_Dataset__Historical_.csv')
print(df_NYC_Clean_Heat_Dataset__Historical_.shape)
df_NYC_Clean_Heat_Dataset__Historical_.head()

(4789, 42)


,"Borough, Block, Lot #",Street Address,Postcode,Borough,Utility,Building Manager,Owner,Owner Address,Owner Telephone,DEP Boiler Application #,#6 Deadline,Boiler Model,# of Identical Boilers,Boiler Capacity (Gross BTU),Boiler Installation Date,Boiler Age Range,Est. Retirement Year,Burner Model,Primary Fuel,Total Gallons (High),Total Gallons (Low),Total MMBTU (High),Total MMBTU (low),Greener Greater Buildings,GGB Deadline,Building Type,Council District,Community Board,Bldg Sqft,# of Bldgs,# of Floors,# of Res. Units,Total Units,Year Built,Condo?,Coop?,Latitude,Longitude,Census Tract,BIN,BBL,NTA
0,1008120001,155 WEST 36 STREET,10018.0,Manhattan,Con Edison,485 7 AVE.ASSOC./COLLIERS,485 SHUR LLC,"485 7 AVENUE#777, MANHATTAN NY 10018",212-971-4000,CA160181H,2012,FEDERAL FST-200,2.0,8.4,1981.0,26 to 30 years old,2016.0,ICI #DEG 84P,#6,131093,91765,13429.08,19184.4,1.0,2022.0,Office Buildings,3.0,M 05,"* 235,450",1.0,15.0,0.0,70.0,1906.0,NaN,NaN,40.751828,-73.988595,109.0,1015235.0,1.008128e+09,Midtown-Midtown South
1,1008340048,330 5 AVENUE,10001.0,Manhattan,Con Edison,SKYLER 330 LLCC/O SHULSKY PROPERTIES INC.,SHULSKY PROPERTIES INC.,"307 FITH AVE, NY NY 10016",212 984-8370,CA323565K,2015,PACIFIC #9303 A,1.0,0.0,1965.0,41 - 45 years old,2010.0,PETRO WD7AH,#4,114975,80483,5075.00,7250.0,1.0,2014.0,Office Buildings,3.0,M 05,"* 145,000",1.0,14.0,0.0,62.0,1926.0,NaN,NaN,40.747521,-73.985239,76.0,1015853.0,1.008340e+09,Midtown-Midtown South
2,1008390009,49 WEST 37 STREET,10018.0,Manhattan,Con Edison,49 W 37 ST REALTY CO,49 W 37TH ST REALTY CO,"440 PARK AVENUE SOUTH, MANHATTAN NY 10016",212 685-6400,CA145582N,0,FEDERAL FST 200,1.0,8.4,1982.0,26 to 30 years old,2017.0,JOHNSON FD68CAHM-200,#4,44640,31248,4687.20,6696.0,1.0,2019.0,Loft Buildings,3.0,M 05,"* 140,898",1.0,18.0,0.0,24.0,1925.0,NaN,NaN,40.751018,-73.984758,84.0,1015958.0,1.008390e+09,Midtown-Midtown South
3,1008670001,411 5 AVENUE,10016.0,Manhattan,Con Edison,ADMS& CO. REAL ESTATE,ADAMS & CO. LLC/FRED LIGUORI,"411 5 AVENUE, MANHATTAN NY 10016",212-679-5500,CA417870Y,0,FEDERAL FST-175,1.0,5.5,1970.0,36 - 40 year old,2010.0,ACE FDL04 SIZE 200,#4,100667,70467,4725.00,6750.0,1.0,2017.0,Office Buildings,3.0,M 05,"* 116,598",1.0,11.0,0.0,1.0,1915.0,NaN,NaN,40.750430,-73.983098,82.0,1017191.0,1.008670e+09,Murray Hill-Kips Bay
4,1022420029,639 WEST 207 STREET,10034.0,Manhattan,Con Edison,WEINER REALTY,PINNACLE WASHINGTON HEIGHTS LLC,"P.O.BO. 1920, NEW YORK NY 10116",NaN,CA068682Y,0,ROCKMILLS MP80,1.0,3.3,1982.0,26 to 30 years old,2017.0,ICI MEG34P,#4,40004,28003,4200.00,6000.0,0.0,NaN,Walk-Up Apartments,7.0,M 12,"* 37,365",1.0,5.0,56.0,58.0,1925.0,NaN,NaN,40.868598,-73.921780,303.0,1064990.0,1.022420e+09,Marble Hill-Inwood


## 60. Buildings Selected for the Heat Sensor Program (HSP)

Every two years beginning in July 2020, the Department of Housing Preservation and Development (HPD) designates 50 class "A" multiple dwellings with heat violations and complaints for participation in a program requiring installation of heat sensors.

https://data.cityofnewyork.us/Housing-Development/Buildings-Selected-for-the-Heat-Sensor-Program-HSP/h4mf-f24e

In [148]:
df_Buildings_Selected_for_the_Heat_Sensor_Program__HSP_= \
    pd.read_csv('data/all/Buildings_Selected_for_the_Heat_Sensor_Program__HSP_.csv')
print(df_Buildings_Selected_for_the_Heat_Sensor_Program__HSP_.shape)
df_Buildings_Selected_for_the_Heat_Sensor_Program__HSP_.head()

(50, 17)


,BUILDING_ID,BOROUGH,NUMBER,STREET,TOTAL_UNITS,PROGRAM_START_DATE,CURRENT_STATUS,DISCHARGE_DATE,Postcode,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,52049,Bronx,968,BRONX PARK SOUTH,28,07/01/2020,Active,NaN,10460,40.845067,-73.878848,206,15,361,2013272,2031340009,East Tremont
1,56597,Bronx,495,CLAREMONT PARKWAY,16,07/01/2020,Active,NaN,10457,40.838595,-73.902015,203,16,167,2009557,2029130043,Claremont-Bathgate
2,60267,Bronx,3212,CRUGER AVENUE,27,07/01/2020,Active,NaN,10467,40.871887,-73.866133,212,15,374,2056262,2045970011,Williamsbridge-Olinville
3,63599,Bronx,340,EAST 148 STREET,9,07/01/2020,Active,NaN,10451,40.815998,-73.920411,201,8,65,2000871,2023290068,Melrose South-Mott Haven North
4,64592,Bronx,361,EAST 163 STREET,59,07/01/2020,Active,NaN,10451,40.826405,-73.914896,204,17,173,2002047,2024240110,East Concourse-Concourse Village


## 61. Inventory of New York City Greenhouse Gas Emissions - Heating And Cooling Degree Days (2004-2016)

Inventory of New York City Greenhouse Gas Emissions - Heating And Cooling Degree Days (2004-2016)

https://data.cityofnewyork.us/Environment/Inventory-of-New-York-City-Greenhouse-Gas-Emission/av3y-hmjv

In [152]:
df_Inventory_of_New_York_City_Greenhouse_Gas_Emissions_Heating_And_Cooling_Degree_Days__2004_2016_= \
    pd.read_csv('data/all/Inventory_of_New_York_City_Greenhouse_Gas_Emissions_-_Heating_And_Cooling_Degree_Days__2004-2016_.csv')
print(df_Inventory_of_New_York_City_Greenhouse_Gas_Emissions_Heating_And_Cooling_Degree_Days__2004_2016_.shape)
df_Inventory_of_New_York_City_Greenhouse_Gas_Emissions_Heating_And_Cooling_Degree_Days__2004_2016_.head()

(26, 9)


,Heating or Cooling Degree Days,Calendar Year,Calendar Year Annual Total,Calendar Year % Change from Previous Year,Calendar Year % Change From Base Year,Fiscal Year,Fiscal Year Annual Total,Fiscal Year % Change from Previous Year,Fiscal Year % Change From Base Year
0,Heating degree days,2004,4787,NaN,NaN,2005.0,4713.0,NaN,NaN
1,Heating degree days,2005,4733,-1.1%,NaN,2006.0,4261.0,-9.6%,NaN
2,Heating degree days,2006,3987,-15.8%,-15.80%,2007.0,4460.0,4.7%,4.70%
3,Heating degree days,2007,4705,18%,-0.60%,2008.0,4470.0,0.2%,4.90%
4,Heating degree days,2008,4598,-2.3%,-2.90%,2009.0,4835.0,8.2%,13.50%


## 62. Hyperlocal Temperature Monitoring

As a part of the Cool Neighborhoods Initiative, the NYC Parks Department, Mayor’s Office of Resilience and NYC Department of Health and Mental Hygiene monitored street level temperature on a subset of city blocks in some of the neighborhoods with highest heat mortality risk during the summers of 2018 and 2019. The dataset includes hourly average values at approximately 475 locations in degrees Fahrenheit.

https://data.cityofnewyork.us/dataset/Hyperlocal-Temperature-Monitoring/qdq3-9eqn


In [154]:
df_Hyperlocal_Temperature_Monitoring= \
    pd.read_csv('data/all/Hyperlocal_Temperature_Monitoring.csv')
print(df_Hyperlocal_Temperature_Monitoring.shape)
df_Hyperlocal_Temperature_Monitoring.head()

(2097150, 10)


,Sensor.ID,AirTemp,Day,Hour,Latitude,Longitude,Year,Install.Type,Borough,ntacode
0,Bk-BR_01,71.189000,06/15/2018,1,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
1,Bk-BR_01,70.243333,06/15/2018,2,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
2,Bk-BR_01,69.392667,06/15/2018,3,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
3,Bk-BR_01,68.263167,06/15/2018,4,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
4,Bk-BR_01,67.114000,06/15/2018,5,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81


## 63. Cool It! NYC 2020 - Cooling Sites

This dataset represents amenities activated as a part of Cool It! NYC, a Citywide plan to increase the amount of cooling features available to the public during heat emergencies, particularly in neighborhoods that face the dangers of high heat. This is part of the Cool It! NYC 2020 Data Collection, which includes the following amenities:

Drinking Fountains: Indicates whether a drinking fountain is activated, not yet activated, broken, or under construction.

Spray Showers: Indicates whether a spray shower installed before July 2020 is activated, not yet activated, broken, or under construction. At this time, spray showers are mapped to the middle of parks.

Cooling Sites: To measure neighborhoods that are the most at risk during extreme heat, NYC Health and Columbia University developed the New York City Heat Vulnerability Index, or HVI. Parks used this data to direct new cooling elements to neighborhoods with HVIs of 4 and 5.

Data Dictionary: https://docs.google.com/spreadsheets/d/1GpXHX9p0e520LcAf3gstOKTQm64wxkdDUiACjhMwd9Q/edit?usp=sharing
        
https://data.cityofnewyork.us/dataset/Cool-It-NYC-2020-Cooling-Sites/h2bn-gu9k

In [156]:
df_Cool_It__NYC_2020_Cooling_Sites= \
    pd.read_csv('data/all/Cool_It__NYC_2020_-_Cooling_Sites.csv')
print(df_Cool_It__NYC_2020_Cooling_Sites.shape)
df_Cool_It__NYC_2020_Cooling_Sites.head()

(273, 10)


,Status,FeatureType,PropertyName,SubPropertyName,GISPropNum,OMPPropID,Borough,District,x,y
0,Activated,Misting Station,Newtown Barge Playground,NaN,B135,B135,Brooklyn,B-01,9.953955e+05,207417.260084
1,Activated,Hydrant Spray Cap,Randall's Island Park,NaN,M107,M107,Manhattan,M-11R,1.002275e+06,224781.691562
2,Activated,Hydrant Spray Cap,Randall's Island Park,NaN,M107,M107,Manhattan,M-11R,1.002487e+06,224686.983747
3,Activated,Misting Station,Rockaway Beach Boardwalk,NaN,Q163,Q163,Queens,Q-14,1.031525e+06,150386.391862
4,Activated,Misting Station,Rockaway Beach Boardwalk,NaN,Q163,Q163,Queens,Q-14,1.031866e+06,150522.134373


## 64. Cool It! NYC 2020 - Spray Showers

This dataset represents amenities activated as a part of Cool It! NYC, a Citywide plan to increase the amount of cooling features available to the public during heat emergencies, particularly in neighborhoods that face the dangers of high heat. This is part of the Cool It! NYC 2020 Data Collection, which includes the following amenities:

Drinking Fountains: Indicates whether a drinking fountain is activated, not yet activated, broken, or under construction.

Spray Showers: Indicates whether a spray shower installed before July 2020 is activated, not yet activated, broken, or under construction. At this time, spray showers are mapped to the middle of parks.

Cooling Sites: To measure neighborhoods that are the most at risk during extreme heat, NYC Health and Columbia University developed the New York City Heat Vulnerability Index, or HVI. Parks used this data to direct new cooling elements to neighborhoods with HVIs of 4 and 5.

Data Dictionary: https://docs.google.com/spreadsheets/d/1GpXHX9p0e520LcAf3gstOKTQm64wxkdDUiACjhMwd9Q/edit?usp=sharing
        
https://data.cityofnewyork.us/dataset/Cool-It-NYC-2020-Spray-Showers/tzuk-eq2f

In [159]:
df_Cool_It__NYC_2020_Spray_Showers= \
    pd.read_csv('data/all/Cool_It__NYC_2020_-_Spray_Showers.csv')
print(df_Cool_It__NYC_2020_Spray_Showers.shape)
df_Cool_It__NYC_2020_Spray_Showers.head()

(755, 12)


,Status,Borough,GISPropNum,OMPPropID,SubPropertyName,PropertyName,Feature,System,FeatureType,District,x,y
0,Activated,B,B001,B001,American Playground,American Playground,Play Equipment,B001-SPRAY01,Sprinkler,B-01,9.959423e+05,204739.176418
1,Under Construction,B,B007,B007-01,Bensonhurst Park Playground,Bensonhurst Park,Play Equipment,B007-01-SPRAY02,Sprinkler,B-11,9.843019e+05,156751.750523
2,Activated,B,B012,B012,Brower Park,Brower Park,Play Equipment,B012-SPRAY03,Sprinkler,B-08,9.999765e+05,184806.013629
3,Activated,B,B016,B016-01,Maria Hernandez Park Playground,Maria Hernandez Park,Play Equipment,B016-01-SPRAY06,Sprinkler,B-04,1.005252e+06,195326.741614
4,Activated,B,B019,B019,Carroll Park,Carroll Park,Play Equipment,B019-SPRAY08,Sprinkler,B-06,9.855052e+05,187419.019569


## 65. Cool It! NYC 2020 - Drinking Fountains


This dataset represents amenities activated as a part of Cool It! NYC, a Citywide plan to increase the amount of cooling features available to the public during heat emergencies, particularly in neighborhoods that face the dangers of high heat. This is part of the Cool It! NYC 2020 Data Collection, which includes the following amenities:

Drinking Fountains: Indicates whether a drinking fountain is activated, not yet activated, broken, or under construction.

Spray Showers: Indicates whether a spray shower installed before July 2020 is activated, not yet activated, broken, or under construction. At this time, spray showers are mapped to the middle of parks.

Cooling Sites: To measure neighborhoods that are the most at risk during extreme heat, NYC Health and Columbia University developed the New York City Heat Vulnerability Index, or HVI. Parks used this data to direct new cooling elements to neighborhoods with HVIs of 4 and 5.

Data Dictionary: https://docs.google.com/spreadsheets/d/1GpXHX9p0e520LcAf3gstOKTQm64wxkdDUiACjhMwd9Q/edit?usp=sharing
        
        
https://data.cityofnewyork.us/dataset/Cool-It-NYC-2020-Drinking-Fountains/wxhr-qbhz

In [161]:
df_Cool_It__NYC_2020_Drinking_Fountains= \
    pd.read_csv('data/all/Cool_It__NYC_2020_-_Drinking_Fountains.csv')
print(df_Cool_It__NYC_2020_Drinking_Fountains.shape)
df_Cool_It__NYC_2020_Drinking_Fountains.head()

(3871, 25)


,System,DF Activated,FountainType,FountainCount,Painted,Filter Installed,Position,Outdoor,GISPropNum,PropertyName,OMPPropID,SubPropertyName,ParkDistrict,Borough,AMPSID,AMPSClass,AMPSStatus,AMPSParentID,AMPSName,CommunityBoard,CouncilDistrict,Precinct,ZIPCode,x,y
0,B001-DF0207,Activated,A,1.0,True,False,"Under Tree, In Playground",True,B001,American Playground,B001,NaN,B-01,B,100037676.0,DRINKFTN,D,B001,A-American Playground,301.0,33.0,94.0,11222.0,9.958536e+05,204780.745672
1,B002-DF0731,Broken,E,1.0,True,False,Out in Open,True,B002,Amersfort Park,B002,NaN,B-18,B,100039159.0,DRINKFTN,I,B002,E-Amersfort Park,318.0,45.0,63.0,11210.0,1.001206e+06,168004.324802
2,B002-DF0732,Broken,E,1.0,True,False,"Under Tree, Out in Open",True,B002,Amersfort Park,B002,NaN,B-18,B,100039177.0,DRINKFTN,I,B002,E-Amersfort Park,318.0,45.0,63.0,11210.0,1.001168e+06,168468.868821
3,B007-DF0754,UnderConstruction,CS Metal,1.0,False,False,In Shade,True,B007,Bensonhurst Park,B007,NaN,B-11,B,100039198.0,DRINKFTN,I,B007,CS Metal-Bensonhurst Park,311.0,43.0,62.0,11214.0,9.842672e+05,157078.860105
4,B007-DF0756,UnderConstruction,A,1.0,True,True,"Out in Open, In Playground",True,B007,Bensonhurst Park,B007-01,Bensonhurst Park Playground,B-11,B,100039249.0,DRINKFTN,I,B007-01,A-Bensonhurst Park-Bensonhurst Park Playground,311.0,43.0,62.0,11214.0,9.843012e+05,156806.697591


## 66. NYCHA Development Data Book

ontains the main body of the “Development Data Book” as of January 1, 2019. The Development Data Book lists all of the Authority's Developments alphabetically and includes information on the development identification numbers, program and construction type, number of apartments and rental rooms, population, number of buildings and stories, street boundaries, and political districts.

https://data.cityofnewyork.us/Housing-Development/NYCHA-Development-Data-Book/evjd-dqpz


In [162]:
df_NYCHA_Development_Data_Book= \
    pd.read_csv('data/all/NYCHA_Development_Data_Book.csv')
print(df_NYCHA_Development_Data_Book.shape)
df_NYCHA_Development_Data_Book.head()

(323, 51)


,DATA AS OF,DEVELOPMENT,HUD AMP#,TDS#,CONSOLIDATED TDS#,DEVELOPMENT EDP#,OPERATING EDP#,HUD #,PROGRAM,METHOD,TYPE,NUMBER OF SECTION 8 TRANSITION APARTMENTS,NUMBER OF CURRENT APARTMENTS,TOTAL NUMBER OF APARTMENTS,NUMBER OF RENTAL ROOMS,AVG NO R/R PER APARTMENT,POPULATION SECTION 8 TRANSITION,POPULATION PUBLIC HOUSING,TOTAL POPULATION,TOTAL # OF FIXED INCOME HOUSEHOLD,PERCENT FIXED INCOME HOUSEHOLDS,NUMBER OF RESIDENTIAL BLDGS,NUMBER OF NON-RESIDENTIAL BLDGS,NUMBER OF STAIRHALLS,NUMBER OF STORIES,TOTAL AREA SQ FT,ACRES,NET DEV AREA SQ FT,EXCLUDING PARK ACRES,BLDG COVERAGE SQ FT,CUBAGE CU FT,BLDG COVERAGE %,DENSITY,DEVELOPMENT COST,PER RENTAL ROOM,AVG MONTHLY GROSS RENT,LOCATION STREET A,LOCATION STREET B,LOCATION STREET C,LOCATION STREET D,BOROUGH,COMMUNITY DISTIRCT,US CONGRESSIONAL DISTRICT,NY STATE SENATE DISTRICT,NY STATE ASSEMBLY DISTRICT,NY CITY COUNCIL DISTRICT,COMPLETION DATE,FEDERALIZED DEVELOPMENT,SENIOR DEVELOPMENT,ELECTRICITY PAID BY RESIDENTS,PRIVATE MANAGEMENT
0,1/1/2019,1010 EAST 178TH STREET,NY005011330,180,180,289,289,NY005090,FEDERAL,CONVENTIONAL,NEW CONST,NaN,218,220,918.0,4.21,NaN,450,450,89.0,0.418,1,0.0,1.0,21,"88,172",2.02,"88,172",2.02,"14,961","1,841,787",16.97%,223.0,"$5,031,405","$5,341",$503,E TREMONT AVE,BRYANT AVE,E 178TH ST,BOSTON RD,BRONX,6,15,32,87,15,3/31/1971,NaN,NaN,NaN,NaN
1,1/1/2019,104-14 TAPSCOTT STREET,NY005011670,242,167,361,283,NY005174,FEDERAL,TURNKEY,NEW CONST,NaN,30,30,131.0,4.37,NaN,65,65,5.0,0.172,1,0.0,1.0,4,"10,000",0.23,"10,000",0.23,"6,983","351,238",69.83%,283.0,"$839,110","$6,505",$643,TAPSCOTT ST,UNION ST,SUTTER AVE,BLAKE AVE,BROOKLYN,16,9,20,55,41,10/31/1972,NaN,NaN,NaN,NaN
2,1/1/2019,1162-1176 WASHINGTON AVENUE,NY005013080,233,308,354,344,NY005138,FEDERAL,TURNKEY,REHAB,NaN,65,66,293.5,4.52,NaN,170,170,21.0,0.328,1,0.0,1.0,6,"18,987",0.44,"18,987",0.44,"12,231","851,926",64.42%,386.0,"$2,205,187","$7,302",$440,E 167TH ST,E 168TH ST,THIRD AVE,WASHINGTON AVE,BRONX,3,15,32,79,16,12/31/1975,NaN,NaN,NaN,NaN
3,1/1/2019,131 SAINT NICHOLAS AVENUE,NY005010970,154,097,264,261,NY005065,FEDERAL,CONVENTIONAL,NEW CONST,NaN,98,100,386.0,3.94,NaN,165,165,41.0,0.427,1,0.0,1.0,17,"29,359",0.67,"29,359",0.67,"5,759","771,591",19.62%,246.0,"$1,880,013","$4,694",$529,SAINT NICHOLAS AVE,W 116TH ST,W 117TH ST,NaN,MANHATTAN,10,13,30,70,09,3/31/1965,NaN,NaN,NaN,NaN
4,1/1/2019,1471 WATSON AVENUE,NY005010670,214,067,332,222,NY005162,FEDERAL,TURNKEY,NEW CONST,NaN,96,96,386.0,4.02,NaN,156,156,47.0,0.505,1,0.0,1.0,6,"39,937",0.92,"39,937",0.92,"13,337","810,629",33.40%,170.0,"$2,278,928","$5,814",$452,WATSON AVE,COLGATE AVE,EVERGREEN AVE,NaN,BRONX,9,15,32,85,17,12/31/1970,NaN,NaN,NaN,NaN
